In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "your_password"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# CSS 스타일 정의
css = """
<style>
    table {
        width: 100%;
        border-collapse: collapse;
    }
    th {
        background-color: blue;
        color: white;
        border: 1px solid gray;
        padding: 5px;
    }
    td {
        border: 1px solid lightgray;
        padding: 5px;
    }
    table, th, td {
        border: 1px solid darkgray;
    }
</style>
"""

# 테이블 생성 함수
def create_html_table(df):
    return css + df.to_html(index=False, escape=False)

# 이메일 본문 템플릿
body_template = """
<p>안녕하세요</p>
<p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
<p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
<h3>[표(요약)]</h3>
{summary_table}
<h3>[신규상세]</h3>
{new_detail_table}
<h3>[해지상세]</h3>
{cancel_detail_table}
<p>감사합니다.<br>장재혁 드림.</p>
"""

# 첨부파일을 포함하여 이메일 전송
for agency, email in dealer_info.items():
    summary_file = os.path.join(output_base_path, f"표(요약)_{agency}.cs")
    new_detail_file = os.path.join(output_base_path, f"신규상세_{agency}.cs")
    cancel_detail_file = os.path.join(output_base_path, f"해지상세_{agency}.cs")

    # 각 파일 읽기
    summary_df = pd.read_csv(summary_file, encoding='utf-8-sig')
    new_detail_df = pd.read_csv(new_detail_file, encoding='utf-8-sig')
    
    summary_table = create_html_table(summary_df)
    new_detail_table = create_html_table(new_detail_df)
    
    # 해지상세 파일이 있을 때만 테이블 생성
    if os.path.exists(cancel_detail_file):
        cancel_detail_df = pd.read_csv(cancel_detail_file, encoding='utf-8-sig')
        cancel_detail_table = create_html_table(cancel_detail_df)
    else:
        cancel_detail_table = "<p>해지상세 데이터가 없습니다.</p>"
    
    subject = f"4월 정산실적_{agency}대리점"
    body = body_template.format(
        summary_table=summary_table,
        new_detail_table=new_detail_table,
        cancel_detail_table=cancel_detail_table
    )
    
    attachments = [summary_file, new_detail_file]
    if os.path.exists(cancel_detail_file):
        attachments.append(cancel_detail_file)
    
    send_email(email, subject, body, attachments)

print("모든 이메일이 성공적으로 전송되었습니다.")


In [32]:
import pandas as pd
import os
import chardet

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 인코딩 감지
summary_encoding = detect_encoding(summary_path)
new_detail_encoding = detect_encoding(new_detail_path)
cancel_detail_encoding = detect_encoding(cancel_detail_path)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = pd.read_csv(summary_path, encoding=summary_encoding, header=6)
df_new_detail = pd.read_csv(new_detail_path, encoding=new_detail_encoding)
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding=cancel_detail_encoding)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월\n(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업\n정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세\n(공급가액10%)', '최종지급액', '비고', '해지환수금액\n(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered.loc[8] = summary_filtered.iloc[0]  # 8행에 값 적용
    summary_filtered = summary_filtered.iloc[0:2]  # 헤더와 첫 데이터 행만 남김
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 필터링된 데이터를 별도 파일로 저장
    summary_output_path = os.path.join(output_base_path, f"표(요약)_{agency}.csv")
    new_detail_output_path = os.path.join(output_base_path, f"신규상세_{agency}.csv")
    cancel_detail_output_path = os.path.join(output_base_path, f"해지상세_{agency}.csv")

    summary_filtered.to_csv(summary_output_path, index=False, encoding='utf-8-sig')
    new_detail_filtered.to_csv(new_detail_output_path, index=False, encoding='utf-8-sig')
    if not cancel_detail_filtered.empty:
        cancel_detail_filtered.to_csv(cancel_detail_output_path, index=False, encoding='utf-8-sig')
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")

print("모든 파일이 처리되고 저장되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/4201696784.py:93: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()


In [34]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

def send_email(to_email, subject, body, attachments=[]):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'html', _charset="utf-8"))

    for attachment in attachments:
        with open(attachment, "rb") as f:
            part = MIMEApplication(f.read(), Name=os.path.basename(attachment))
            part['Content-Disposition'] = f'attachment; filename="{os.path.basename(attachment)}"'
            msg.attach(part)

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, to_email, msg.as_string())
        print(f"{to_email}로 이메일을 성공적으로 보냈습니다.")
    except Exception as e:
        print(f"{to_email}로 이메일을 보내는 데 실패했습니다: {e}")

# 이메일 본문 템플릿
body_template = """
<p>안녕하세요</p>
<p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
<p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
<h3>[표(요약)]</h3>
<p>파일 첨부</p>
<h3>[신규상세]</h3>
<p>파일 첨부</p>
<h3>[해지상세]</h3>
<p>파일 첨부 (해지상세 데이터가 있는 경우)</p>
<p>감사합니다.<br>장재혁 드림.</p>
"""

# 첨부파일을 포함하여 이메일 전송
for agency, email in dealer_info.items():
    attachments = [
        os.path.join(output_base_path, f"표(요약)_{agency}.cs"),
        os.path.join(output_base_path, f"신규상세_{agency}.cs")
    ]
    
    cancel_detail_file = os.path.join(output_base_path, f"해지상세_{agency}.cs")
    if os.path.exists(cancel_detail_file):
        attachments.append(cancel_detail_file)
    
    subject = f"4월 정산실적_{agency}대리점"
    body = body_template
    send_email(email, subject, body, attachments)

print("모든 이메일이 성공적으로 전송되었습니다.")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/heebonpark/Downloads/4월 정산 전송/표(요약)_(주)미래넷.cs'

In [36]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 인코딩 감지
summary_encoding = detect_encoding(summary_path)
new_detail_encoding = detect_encoding(new_detail_path)
cancel_detail_encoding = detect_encoding(cancel_detail_path)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = pd.read_csv(summary_path, encoding=summary_encoding, header=6)
df_new_detail = pd.read_csv(new_detail_path, encoding=new_detail_encoding)
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding=cancel_detail_encoding)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월\n(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업\n정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세\n(공급가액10%)', '최종지급액', '비고', '해지환수금액\n(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"
smtp_server = "smtp.naver.com"
smtp_port = 587

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

def send_email(receiver_email, subject, body, attachments):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    
    msg.attach(MIMEText(body, 'html'))
    
    for attachment in attachments:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(open(attachment, 'rb').read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="{}"'.format(os.path.basename(attachment)))
        msg.attach(part)
    
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered.loc[8] = summary_filtered.iloc[0]  # 8행에 값 적용
    summary_filtered = summary_filtered.iloc[0:2]  # 헤더와 첫 데이터 행만 남김
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 필터링된 데이터를 별도 파일로 저장
    summary_output_path = os.path.join(output_base_path, f"표(요약)_{agency}.csv")
    new_detail_output_path = os.path.join(output_base_path, f"신규상세_{agency}.csv")
    cancel_detail_output_path = os.path.join(output_base_path, f"해지상세_{agency}.csv")

    summary_filtered.to_csv(summary_output_path, index=False, encoding='utf-8-sig')
    new_detail_filtered.to_csv(new_detail_output_path, index=False, encoding='utf-8-sig')
    if not cancel_detail_filtered.empty:
        cancel_detail_filtered.to_csv(cancel_detail_output_path, index=False, encoding='utf-8-sig')
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")
    
    # 이메일 보내기
    receiver_email = dealer_info[agency]
    subject = f"4월 정산 내역 - {agency}"
    body = f"""
    <html>
    <head>
        <style>
            table {{
                width: 100%;
                border-collapse: collapse;
            }}
            th {{
                background-color: blue;
                color: white;
                border: 1px solid gray;
                padding: 5px;
                text-align: center;
            }}
            td {{
                border: 1px solid gray;
                padding: 5px;
                text-align: center;
            }}
            .outer {{
                border: 2px solid darkgray;
                padding: 10px;
            }}
        </style>
    </head>
    <body>
        <div class="outer">
            <p>안녕하세요,</p>
            <p>{agency}의 4월 정산 내역을 첨부합니다.</p>
            <table>
                <tr>
                    <th>정산년월(YYMM)</th><th>본부</th><th>지사</th><th>추천자사번</th><th>대표자</th><th>상호명</th><th>사업자등록번호</th>
                    <th>신규월정료</th><th>신규건수</th><th>모집수수료</th><th>성사율수수료</th><th>완결비중수수료</th><th>완결영업정책수수료</th>
                    <th>프로모션</th><th>합계</th><th>유지수수료</th><th>판매형</th><th>소급액</th><th>전월미지급액</th><th>전월미환수</th>
                    <th>공급가액</th><th>부가세(공급가액10%)</th><th>최종지급액</th><th>비고</th><th>해지환수금액(별도세금계산서발행)</th><th>승인번호</th>
                </tr>
                <tr>
                    <td>{summary_filtered.iloc[1]['정산년월\n(YYMM)']}</td><td>{summary_filtered.iloc[1]['본부']}</td><td>{summary_filtered.iloc[1]['지사']}</td>
                    <td>{summary_filtered.iloc[1]['추천자사번']}</td><td>{summary_filtered.iloc[1]['대표자']}</td><td>{summary_filtered.iloc[1]['상호명']}</td>
                    <td>{summary_filtered.iloc[1]['사업자등록번호']}</td><td>{summary_filtered.iloc[1]['신규월정료']}</td><td>{summary_filtered.iloc[1]['신규건수']}</td>
                    <td>{summary_filtered.iloc[1]['모집수수료']}</td><td>{summary_filtered.iloc[1]['성사율수수료']}</td><td>{summary_filtered.iloc[1]['완결비중수수료']}</td>
                    <td>{summary_filtered.iloc[1]['완결영업\n정책수수료']}</td><td>{summary_filtered.iloc[1]['프로모션']}</td><td>{summary_filtered.iloc[1]['합계']}</td>
                    <td>{summary_filtered.iloc[1]['유지수수료']}</td><td>{summary_filtered.iloc[1]['판매형']}</td><td>{summary_filtered.iloc[1]['소급액']}</td>
                    <td>{summary_filtered.iloc[1]['전월미지급액']}</td><td>{summary_filtered.iloc[1]['전월미환수']}</td><td>{summary_filtered.iloc[1]['공급가액']}</td>
                    <td>{summary_filtered.iloc[1]['부가세\n(공급가액10%)']}</td><td>{summary_filtered.iloc[1]['최종지급액']}</td><td>{summary_filtered.iloc[1]['비고']}</td>
                    <td>{summary_filtered.iloc[1]['해지환수금액\n(별도세금계산서발행)']}</td><td>{summary_filtered.iloc[1]['승인번호']}</td>
                </tr>
            </table>
            <p>감사합니다.</p>
        </div>
    </body>
    </html>
    """
    attachments = [summary_output_path, new_detail_output_path]
    if not cancel_detail_filtered.empty:
        attachments.append(cancel_detail_output_path)
    
    send_email(receiver_email, subject, body, attachments)

print("모든 파일이 처리되고 저장되었습니다.")


SyntaxError: f-string expression part cannot include a backslash (3208574185.py, line 220)

In [37]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 인코딩 감지
summary_encoding = detect_encoding(summary_path)
new_detail_encoding = detect_encoding(new_detail_path)
cancel_detail_encoding = detect_encoding(cancel_detail_path)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = pd.read_csv(summary_path, encoding=summary_encoding, header=6)
df_new_detail = pd.read_csv(new_detail_path, encoding=new_detail_encoding)
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding=cancel_detail_encoding)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월\n(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업\n정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세\n(공급가액10%)', '최종지급액', '비고', '해지환수금액\n(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"
smtp_server = "smtp.naver.com"
smtp_port = 587

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

def send_email(receiver_email, subject, body, attachments):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    
    msg.attach(MIMEText(body, 'html'))
    
    for attachment in attachments:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(open(attachment, 'rb').read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="{}"'.format(os.path.basename(attachment)))
        msg.attach(part)
    
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered.loc[8] = summary_filtered.iloc[0]  # 8행에 값 적용
    summary_filtered = summary_filtered.iloc[0:2]  # 헤더와 첫 데이터 행만 남김
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 필터링된 데이터를 별도 파일로 저장
    summary_output_path = os.path.join(output_base_path, f"표(요약)_{agency}.csv")
    new_detail_output_path = os.path.join(output_base_path, f"신규상세_{agency}.csv")
    cancel_detail_output_path = os.path.join(output_base_path, f"해지상세_{agency}.csv")

    summary_filtered.to_csv(summary_output_path, index=False, encoding='utf-8-sig')
    new_detail_filtered.to_csv(new_detail_output_path, index=False, encoding='utf-8-sig')
    if not cancel_detail_filtered.empty:
        cancel_detail_filtered.to_csv(cancel_detail_output_path, index=False, encoding='utf-8-sig')
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")
    
    # 이메일 보내기
    receiver_email = dealer_info[agency]
    subject = f"4월 정산 내역 - {agency}"
    body = f"""
    <html>
    <head>
        <style>
            table {{
                width: 100%;
                border-collapse: collapse;
            }}
            th {{
                background-color: blue;
                color: white;
                border: 1px solid gray;
                padding: 5px;
                text-align: center;
            }}
            td {{
                border: 1px solid gray;
                padding: 5px;
                text-align: center;
            }}
            .outer {{
                border: 2px solid darkgray;
                padding: 10px;
            }}
        </style>
    </head>
    <body>
        <div class="outer">
            <p>안녕하세요,</p>
            <p>{agency}의 4월 정산 내역을 첨부합니다.</p>
            <table>
                <tr>
                    <th>정산년월(YYMM)</th><th>본부</th><th>지사</th><th>추천자사번</th><th>대표자</th><th>상호명</th><th>사업자등록번호</th>
                    <th>신규월정료</th><th>신규건수</th><th>모집수수료</th><th>성사율수수료</th><th>완결비중수수료</th><th>완결영업정책수수료</th>
                    <th>프로모션</th><th>합계</th><th>유지수수료</th><th>판매형</th><th>소급액</th><th>전월미지급액</th><th>전월미환수</th>
                    <th>공급가액</th><th>부가세(공급가액10%)</th><th>최종지급액</th><th>비고</th><th>해지환수금액(별도세금계산서발행)</th><th>승인번호</th>
                </tr>
                <tr>
                    <td>{summary_filtered.iloc[1]['정산년월\n(YYMM)']}</td><td>{summary_filtered.iloc[1]['본부']}</td><td>{summary_filtered.iloc[1]['지사']}</td>
                    <td>{summary_filtered.iloc[1]['추천자사번']}</td><td>{summary_filtered.iloc[1]['대표자']}</td><td>{summary_filtered.iloc[1]['상호명']}</td>
                    <td>{summary_filtered.iloc[1]['사업자등록번호']}</td><td>{summary_filtered.iloc[1]['신규월정료']}</td><td>{summary_filtered.iloc[1]['신규건수']}</td>
                    <td>{summary_filtered.iloc[1]['모집수수료']}</td><td>{summary_filtered.iloc[1]['성사율수수료']}</td><td>{summary_filtered.iloc[1]['완결비중수수료']}</td>
                    <td>{summary_filtered.iloc[1]['완결영업\n정책수수료']}</td><td>{summary_filtered.iloc[1]['프로모션']}</td><td>{summary_filtered.iloc[1]['합계']}</td>
                    <td>{summary_filtered.iloc[1]['유지수수료']}</td><td>{summary_filtered.iloc[1]['판매형']}</td><td>{summary_filtered.iloc[1]['소급액']}</td>
                    <td>{summary_filtered.iloc[1]['전월미지급액']}</td><td>{summary_filtered.iloc[1]['전월미환수']}</td><td>{summary_filtered.iloc[1]['공급가액']}</td>
                    <td>{summary_filtered.iloc[1]['부가세\n(공급가액10%)']}</td><td>{summary_filtered.iloc[1]['최종지급액']}</td><td>{summary_filtered.iloc[1]['비고']}</td>
                    <td>{summary_filtered.iloc[1]['해지환수금액\n(별도세금계산서발행)']}</td><td>{summary_filtered.iloc[1]['승인번호']}</td>
                </tr>
            </table>
            <p>감사합니다.</p>
        </div>
    </body>
    </html>
    """
    attachments = [summary_output_path, new_detail_output_path]
    if not cancel_detail_filtered.empty:
        attachments.append(cancel_detail_output_path)
    
    send_email(receiver_email, subject, body, attachments)

print("모든 파일이 처리되고 저장되었습니다.")


SyntaxError: f-string expression part cannot include a backslash (3208574185.py, line 220)

In [38]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 인코딩 감지
summary_encoding = detect_encoding(summary_path)
new_detail_encoding = detect_encoding(new_detail_path)
cancel_detail_encoding = detect_encoding(cancel_detail_path)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = pd.read_csv(summary_path, encoding=summary_encoding, header=6)
df_new_detail = pd.read_csv(new_detail_path, encoding=new_detail_encoding)
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding=cancel_detail_encoding)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월\n(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업\n정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세\n(공급가액10%)', '최종지급액', '비고', '해지환수금액\n(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"
smtp_server = "smtp.naver.com"
smtp_port = 587

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

def send_email(receiver_email, subject, body, attachments):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    
    msg.attach(MIMEText(body, 'html'))
    
    for attachment in attachments:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(open(attachment, 'rb').read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="{}"'.format(os.path.basename(attachment)))
        msg.attach(part)
    
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered.loc[8] = summary_filtered.iloc[0]  # 8행에 값 적용
    summary_filtered = summary_filtered.iloc[0:2]  # 헤더와 첫 데이터 행만 남김
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 필터링된 데이터를 별도 파일로 저장
    summary_output_path = os.path.join(output_base_path, f"표(요약)_{agency}.csv")
    new_detail_output_path = os.path.join(output_base_path, f"신규상세_{agency}.csv")
    cancel_detail_output_path = os.path.join(output_base_path, f"해지상세_{agency}.csv")

    summary_filtered.to_csv(summary_output_path, index=False, encoding='utf-8-sig')
    new_detail_filtered.to_csv(new_detail_output_path, index=False, encoding='utf-8-sig')
    if not cancel_detail_filtered.empty:
        cancel_detail_filtered.to_csv(cancel_detail_output_path, index=False, encoding='utf-8-sig')
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")
    
    # 이메일 보내기
    receiver_email = dealer_info[agency]
    subject = f"4월 정산 내역 - {agency}"
    body = f"""
    <html>
    <head>
        <style>
            table {{
                width: 100%;
                border-collapse: collapse;
            }}
            th {{
                background-color: blue;
                color: white;
                border: 1px solid gray;
                padding: 5px;
                text-align: center;
            }}
            td {{
                border: 1px solid gray;
                padding: 5px;
                text-align: center;
            }}
            .outer {{
                border: 2px solid darkgray;
                padding: 10px;
            }}
        </style>
    </head>
    <body>
        <div class="outer">
            <p>안녕하세요,</p>
            <p>{agency}의 4월 정산 내역을 첨부합니다.</p>
            <table>
                <tr>
                    <th>정산년월(YYMM)</th><th>본부</th><th>지사</th><th>추천자사번</th><th>대표자</th><th>상호명</th><th>사업자등록번호</th>
                    <th>신규월정료</th><th>신규건수</th><th>모집수수료</th><th>성사율수수료</th><th>완결비중수수료</th><th>완결영업정책수수료</th>
                    <th>프로모션</th><th>합계</th><th>유지수수료</th><th>판매형</th><th>소급액</th><th>전월미지급액</th><th>전월미환수</th>
                    <th>공급가액</th><th>부가세(공급가액10%)</th><th>최종지급액</th><th>비고</th><th>해지환수금액(별도세금계산서발행)</th><th>승인번호</th>
                </tr>
                <tr>
                    <td>{summary_filtered.iloc[1]['정산년월\n(YYMM)']}</td><td>{summary_filtered.iloc[1]['본부']}</td><td>{summary_filtered.iloc[1]['지사']}</td>
                    <td>{summary_filtered.iloc[1]['추천자사번']}</td><td>{summary_filtered.iloc[1]['대표자']}</td><td>{summary_filtered.iloc[1]['상호명']}</td>
                    <td>{summary_filtered.iloc[1]['사업자등록번호']}</td><td>{summary_filtered.iloc[1]['신규월정료']}</td><td>{summary_filtered.iloc[1]['신규건수']}</td>
                    <td>{summary_filtered.iloc[1]['모집수수료']}</td><td>{summary_filtered.iloc[1]['성사율수수료']}</td><td>{summary_filtered.iloc[1]['완결비중수수료']}</td>
                    <td>{summary_filtered.iloc[1]['완결영업\n정책수수료']}</td><td>{summary_filtered.iloc[1]['프로모션']}</td><td>{summary_filtered.iloc[1]['합계']}</td>
                    <td>{summary_filtered.iloc[1]['유지수수료']}</td><td>{summary_filtered.iloc[1]['판매형']}</td><td>{summary_filtered.iloc[1]['소급액']}</td>
                    <td>{summary_filtered.iloc[1]['전월미지급액']}</td><td>{summary_filtered.iloc[1]['전월미환수']}</td><td>{summary_filtered.iloc[1]['공급가액']}</td>
                    <td>{summary_filtered.iloc[1]['부가세\n(공급가액10%)']}</td><td>{summary_filtered.iloc[1]['최종지급액']}</td><td>{summary_filtered.iloc[1]['비고']}</td>
                    <td>{summary_filtered.iloc[1]['해지환수금액\n(별도세금계산서발행)']}</td><td>{summary_filtered.iloc[1]['승인번호']}</td>
                </tr>
            </table>
            <p>감사합니다.</p>
        </div>
    </body>
    </html>
    """
    attachments = [summary_output_path, new_detail_output_path]
    if not cancel_detail_filtered.empty:
        attachments.append(cancel_detail_output_path)
    
    send_email(receiver_email, subject, body, attachments)

print("모든 파일이 처리되고 저장되었습니다.")


SyntaxError: f-string expression part cannot include a backslash (3208574185.py, line 220)

In [39]:
import pandas as pd
import os
import chardet

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 인코딩 감지
summary_encoding = detect_encoding(summary_path)
new_detail_encoding = detect_encoding(new_detail_path)
cancel_detail_encoding = detect_encoding(cancel_detail_path)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = pd.read_csv(summary_path, encoding=summary_encoding, header=6)
df_new_detail = pd.read_csv(new_detail_path, encoding=new_detail_encoding)
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding=cancel_detail_encoding)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월\n(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업\n정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세\n(공급가액10%)', '최종지급액', '비고', '해지환수금액\n(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered.loc[8] = summary_filtered.iloc[0]  # 8행에 값 적용
    summary_filtered = summary_filtered.iloc[0:2]  # 헤더와 첫 데이터 행만 남김
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 필터링된 데이터를 별도 파일로 저장
    summary_output_path = os.path.join(output_base_path, f"표(요약)_{agency}.csv")
    new_detail_output_path = os.path.join(output_base_path, f"신규상세_{agency}.csv")
    cancel_detail_output_path = os.path.join(output_base_path, f"해지상세_{agency}.csv")

    summary_filtered.to_csv(summary_output_path, index=False, encoding='utf-8-sig')
    new_detail_filtered.to_csv(new_detail_output_path, index=False, encoding='utf-8-sig')
    if not cancel_detail_filtered.empty:
        cancel_detail_filtered.to_csv(cancel_detail_output_path, index=False, encoding='utf-8-sig')
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/1243703365.py:104: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()


In [40]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# HTML 템플릿 설정
html_template = """
<html>
<head>
    <style>
        table {{
            width: 100%;
            border-collapse: collapse;
        }}
        th {{
            background-color: #007BFF;
            color: white;
            border: 1px solid gray;
            padding: 8px;
            text-align: center;
        }}
        td {{
            border: 1px solid gray;
            padding: 8px;
            text-align: center;
        }}
    </style>
</head>
<body>
    <p>안녕하세요,</p>
    <p>첨부된 파일은 {agency}에 대한 2024년 4월 정산 내역입니다.</p>
    <table>
        <thead>
            <tr>
                <th>정산년월(YYMM)</th>
                <th>본부</th>
                <th>지사</th>
                <th>추천자사번</th>
                <th>대표자</th>
                <th>상호명</th>
                <th>사업자등록번호</th>
                <th>신규월정료</th>
                <th>신규건수</th>
                <th>모집수수료</th>
                <th>성사율수수료</th>
                <th>완결비중수수료</th>
                <th>완결정책수수료</th>
                <th>프로모션</th>
                <th>합계</th>
                <th>유지수수료</th>
                <th>판매형</th>
                <th>소급액</th>
                <th>전월미지급액</th>
                <th>전월미환수</th>
                <th>공급가액</th>
                <th>부가세(공급가액10%)</th>
                <th>최종지급액</th>
                <th>비고</th>
                <th>해지환수금액(별도세금계산서발행)</th>
                <th>승인번호</th>
            </tr>
        </thead>
        <tbody>
            {table_rows}
        </tbody>
    </table>
    <p>감사합니다.</p>
</body>
</html>
"""

# 각 대리점에 대해 이메일 전송
for agency in agencies:
    if agency not in dealer_info:
        print(f"{agency}의 이메일 정보가 없습니다.")
        continue

    receiver_email = dealer_info[agency]
    
    # 이메일 메시지 작성
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"{agency} - 2024년 4월 정산 내역"

    # 테이블 행 생성
    table_rows = ""
    for _, row in summary_filtered.iterrows():
        table_rows += "<tr>" + "".join([f"<td>{cell}</td>" for cell in row]) + "</tr>"

    # HTML 메시지 생성
    html_message = html_template.format(agency=agency, table_rows=table_rows)
    msg.attach(MIMEText(html_message, 'html'))

    # 파일 첨부
    attachments = [
        os.path.join(output_base_path, f"표(요약)_{agency}.csv"),
        os.path.join(output_base_path, f"신규상세_{agency}.csv"),
        os.path.join(output_base_path, f"해지상세_{agency}.csv")
    ]

    for file_path in attachments:
        if os.path.exists(file_path):
            with open(file_path, "rb") as attachment:
                part = MIMEBase("application", "octet-stream")
                part.set_payload(attachment.read())
                encoders.encode_base64(part)
                part.add_header(
                    "Content-Disposition",
                    f"attachment; filename= {os.path.basename(file_path)}",
                )
                msg.attach(part)

    # 이메일 전송
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
            print(f"{agency}에 대한 이메일이 성공적으로 전송되었습니다.")
    except Exception as e:
        print(f"{agency}에 대한 이메일 전송 중 오류 발생: {e}")


(주)미래넷에 대한 이메일이 성공적으로 전송되었습니다.
구정례에 대한 이메일이 성공적으로 전송되었습니다.
케이티텔보안네트워크에 대한 이메일이 성공적으로 전송되었습니다.
커넥션네트워크에 대한 이메일이 성공적으로 전송되었습니다.
애드라인에 대한 이메일이 성공적으로 전송되었습니다.
제제컴퍼니에 대한 이메일이 성공적으로 전송되었습니다.
아이티인프라에 대한 이메일이 성공적으로 전송되었습니다.


In [42]:
import pandas as pd
import os

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 데이터 읽기
df_summary = pd.read_csv(summary_path, encoding='euc-kr', skiprows=6)
df_new_detail = pd.read_csv(new_detail_path, encoding='euc-kr')
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding='euc-kr')

# 필요 없는 열 삭제 및 헤더 설정
df_summary = df_summary.drop(columns=[col for col in df_summary.columns if 'Unnamed' in col])
df_summary.columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호', '신규월정료',
    '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결정책수수료', '프로모션', '합계', '유지수수료',
    '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고',
    '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 중복 행 제거
df_summary = df_summary.drop_duplicates()

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 필터링된 데이터 저장 경로 설정
output_base_path = "/mnt/data/4월 정산 전송/"

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

for agency in agencies:
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터 저장
    summary_filtered.to_csv(os.path.join(output_base_path, f"표(요약)_{agency}.csv"), index=False, encoding='euc-kr')
    new_detail_filtered.to_csv(os.path.join(output_base_path, f"신규상세_{agency}.csv"), index=False, encoding='euc-kr')
    cancel_detail_filtered.to_csv(os.path.join(output_base_path, f"해지상세_{agency}.csv"), index=False, encoding='euc-kr')

print("모든 파일이 처리되고 저장되었습니다.")


UnicodeDecodeError: 'euc_kr' codec can't decode byte 0x8e in position 107090: illegal multibyte sequence

In [43]:
import pandas as pd
import os

# 파일 경로 설정
# base_path = "/mnt/data/"

base_path = r"/Users/heebonpark/Downloads"
summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 데이터 읽기
df_summary = pd.read_csv(summary_path, encoding='euc-kr', skiprows=6)
df_new_detail = pd.read_csv(new_detail_path, encoding='euc-kr')
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding='euc-kr')

# 필요 없는 열 삭제 및 헤더 설정
df_summary = df_summary.drop(columns=[col for col in df_summary.columns if 'Unnamed' in col])
df_summary.columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호', '신규월정료',
    '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결정책수수료', '프로모션', '합계', '유지수수료',
    '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고',
    '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 중복 행 제거
df_summary = df_summary.drop_duplicates()

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 필터링된 데이터 저장 경로 설정
output_base_path = "/mnt/data/4월 정산 전송/"

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

for agency in agencies:
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터 저장
    summary_filtered.to_csv(os.path.join(output_base_path, f"표(요약)_{agency}.csv"), index=False, encoding='euc-kr')
    new_detail_filtered.to_csv(os.path.join(output_base_path, f"신규상세_{agency}.csv"), index=False, encoding='euc-kr')
    cancel_detail_filtered.to_csv(os.path.join(output_base_path, f"해지상세_{agency}.csv"), index=False, encoding='euc-kr')

print("모든 파일이 처리되고 저장되었습니다.")


UnicodeDecodeError: 'euc_kr' codec can't decode byte 0x8e in position 107090: illegal multibyte sequence

In [44]:
import pandas as pd
import os

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 데이터 읽기
df_summary = pd.read_csv(summary_path, encoding='euc-kr', skiprows=6)

# 새로운 시도: utf-8 인코딩 사용
try:
    df_new_detail = pd.read_csv(new_detail_path, encoding='utf-8')
except UnicodeDecodeError:
    print("utf-8 인코딩 실패, euc-kr 인코딩을 시도합니다.")
    df_new_detail = pd.read_csv(new_detail_path, encoding='euc-kr')

try:
    df_cancel_detail = pd.read_csv(cancel_detail_path, encoding='utf-8')
except UnicodeDecodeError:
    print("utf-8 인코딩 실패, euc-kr 인코딩을 시도합니다.")
    df_cancel_detail = pd.read_csv(cancel_detail_path, encoding='euc-kr')

# 필요 없는 열 삭제 및 헤더 설정
df_summary = df_summary.drop(columns=[col for col in df_summary.columns if 'Unnamed' in col])
df_summary.columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호', '신규월정료',
    '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결정책수수료', '프로모션', '합계', '유지수수료',
    '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고',
    '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 중복 행 제거
df_summary = df_summary.drop_duplicates()

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 필터링된 데이터 저장 경로 설정
output_base_path = "/mnt/data/4월 정산 전송/"

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

for agency in agencies:
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터 저장
    summary_filtered.to_csv(os.path.join(output_base_path, f"표(요약)_{agency}.csv"), index=False, encoding='euc-kr')
    new_detail_filtered.to_csv(os.path.join(output_base_path, f"신규상세_{agency}.csv"), index=False, encoding='euc-kr')
    cancel_detail_filtered.to_csv(os.path.join(output_base_path, f"해지상세_{agency}.csv"), index=False, encoding='euc-kr')

print("모든 파일이 처리되고 저장되었습니다.")


utf-8 인코딩 실패, euc-kr 인코딩을 시도합니다.


UnicodeDecodeError: 'euc_kr' codec can't decode byte 0x8e in position 107090: illegal multibyte sequence

In [45]:
import pandas as pd
import os
import chardet

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 인코딩 감지
summary_encoding = detect_encoding(summary_path)
new_detail_encoding = detect_encoding(new_detail_path)
cancel_detail_encoding = detect_encoding(cancel_detail_path)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = pd.read_csv(summary_path, encoding=summary_encoding, header=6)
df_new_detail = pd.read_csv(new_detail_path, encoding=new_detail_encoding)
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding=cancel_detail_encoding)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월\n(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업\n정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세\n(공급가액10%)', '최종지급액', '비고', '해지환수금액\n(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 별도 파일로 저장
    summary_output_path = os.path.join(output_base_path, f"표(요약)_{agency}.csv")
    new_detail_output_path = os.path.join(output_base_path, f"신규상세_{agency}.csv")
    cancel_detail_output_path = os.path.join(output_base_path, f"해지상세_{agency}.csv")

    summary_filtered.to_csv(summary_output_path, index=False, encoding='utf-8-sig')
    new_detail_filtered.to_csv(new_detail_output_path, index=False, encoding='utf-8-sig')
    if not cancel_detail_filtered.empty:
        cancel_detail_filtered.to_csv(cancel_detail_output_path, index=False, encoding='utf-8-sig')
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")

print("모든 파일이 처리되고 저장되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/459205472.py:102: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()


In [46]:
import pandas as pd
import os
import chardet

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 인코딩 감지
summary_encoding = detect_encoding(summary_path)
new_detail_encoding = detect_encoding(new_detail_path)
cancel_detail_encoding = detect_encoding(cancel_detail_path)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = pd.read_csv(summary_path, encoding=summary_encoding, header=6)
df_new_detail = pd.read_csv(new_detail_path, encoding=new_detail_encoding)
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding=cancel_detail_encoding)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월\n(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업\n정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세\n(공급가액10%)', '최종지급액', '비고', '해지환수금액\n(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 하나의 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")

print("모든 파일이 처리되고 저장되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/722180783.py:102: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()


In [47]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# 파일 경로 설정
base_path = r"/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 인코딩 감지
summary_encoding = detect_encoding(summary_path)
new_detail_encoding = detect_encoding(new_detail_path)
cancel_detail_encoding = detect_encoding(cancel_detail_path)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = pd.read_csv(summary_path, encoding=summary_encoding, header=6)
df_new_detail = pd.read_csv(new_detail_path, encoding=new_detail_encoding)
df_cancel_detail = pd.read_csv(cancel_detail_path, encoding=cancel_detail_encoding)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월\n(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업\n정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세\n(공급가액10%)', '최종지급액', '비고', '해지환수금액\n(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 하나의 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")
    
    # 이메일 전송
    receiver_email = dealer_info[agency]
    
    # 이메일 메시지 설정
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"{agency} 4월 정산 파일 전송"

    # 이메일 본문 설정
    body = f"""
    <html>
    <body>
        <p>안녕하세요,<br><br>
        {agency}의 4월 정산 파일을 첨부드립니다.<br><br>
        감사합니다.<br>
        KT 팀 드림</p>
    </body>
    </html>
    """
    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="xlsx")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)
    
    # SMTP 서버 연결 및 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
    
    print(f"{agency}에 대한 이메일이 전송되었습니다.")

print("모든 파일이 처리되고 저장되었으며, 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/1273703449.py:123: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()


구정례에 대한 이메일이 전송되었습니다.
케이티텔보안네트워크에 대한 파일을 처리하고 저장했습니다.
케이티텔보안네트워크에 대한 이메일이 전송되었습니다.
커넥션네트워크에 대한 파일을 처리하고 저장했습니다.
커넥션네트워크에 대한 이메일이 전송되었습니다.
애드라인에 대한 파일을 처리하고 저장했습니다.
애드라인에 대한 이메일이 전송되었습니다.
제제컴퍼니에 대한 파일을 처리하고 저장했습니다.
제제컴퍼니에 대한 이메일이 전송되었습니다.
아이티인프라에 대한 파일을 처리하고 저장했습니다.
아이티인프라에 대한 이메일이 전송되었습니다.
모든 파일이 처리되고 저장되었으며, 이메일이 전송되었습니다.


In [55]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 하나의 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")
    
    # 이메일 전송
    receiver_email = dealer_info[agency]
    
    # 이메일 메시지 설정
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"{agency} 4월 정산 파일 전송"
    
    # 이메일 본문 설정
    body = f"""
    <html>
    <body>
        <p>안녕하세요, {agency} 님.</p>
        <p>첨부된 파일은 4월 정산 내역입니다. 확인 부탁드립니다.</p>
        <p>보내는 사람: 장재혁</p>
        <table border="1" style="border-collapse: collapse;">
            <tr>
                <th>정산년월(YYMM)</th>
                <th>본부</th>
                <th>지사</th>
                <th>추천자사번</th>
                <th>대표자</th>
                <th>상호명</th>
                <th>사업자등록번호</th>
                <th>신규월정료</th>
                <th>신규건수</th>
                <th>모집수수료</th>
                <th>성사율수수료</th>
                <th>완결비중수수료</th>
                <th>완결정책수수수료</th>
                <th>프로모션</th>
                <th>합계</th>
                <th>유지수수료</th>
                <th>판매형</th>
                <th>소급액</th>
                <th>전월미지급액</th>
                <th>전월미환수</th>
                <th>공급가액</th>
                <th>부가세(공급가액10%)</th>
                <th>최종지급액</th>
                <th>비고</th>
                <th>해지환수금액(별도세금계산서발행)</th>
                <th>승인번호</th>
            </tr>
            <tr>
                {''.join([f'<td>{cell}</td>' for cell in summary_filtered.iloc[0]])}
            </tr>
        </table>
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

KeyError: "['정산년월(YYMM)', '완결영업정책수수료', '부가세(공급가액10%)', '해지환수금액(별도세금계산서발행)'] not in index"

In [58]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
# 필요한 열 이름 업데이트
required_columns = [
    '정산년월\n(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업\n정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세\n(공급가액10%)', '최종지급액', '비고', '해지환수금액\n(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 하나의 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")
    
    # 이메일 전송
    receiver_email = dealer_info[agency]
    
    # 이메일 메시지 설정
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"{agency} 4월 정산 파일 전송"
    
    # 이메일 본문 설정
    body = f"""
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>첨부된 파일은 4월 정산 내역입니다. 확인 부탁드립니다.</p>
        <p>보내는 사람: 장재혁</p>
        <table border="1" style="border-collapse: collapse;">
            <tr>
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            <tr>
                {''.join([f'<td>{cell}</td>' for cell in summary_filtered.iloc[0]])}
            </tr>
        </table>
    </body>
    </html>
    """
    
    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

KeyError: "['정산년월\\n(YYMM)', '완결영업\\n정책수수료', '부가세\\n(공급가액10%)', '해지환수금액\\n(별도세금계산서발행)'] not in index"

In [59]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 하나의 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")
    
    # 이메일 전송
    receiver_email = dealer_info[agency]
    
    # 이메일 메시지 설정
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"{agency} 4월 정산 파일 전송"
    
    # 이메일 본문 설정
    body = f"""
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>첨부된 파일은 4월 정산 내역입니다. 확인 부탁드립니다.</p>
        <p>보내는 사람: 장재혁</p>
        <table border="1" style="border-collapse: collapse;">
            <tr>
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            <tr>
                {''.join([f'<td>{cell}</td>' for cell in summary_filtered.iloc[0]])}
            </tr>
        </table>
    </body>
    </html>
    """
    
    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/80075376.py:128: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()


구정례에 대한 이메일을 bough38@naver.com로 전송했습니다.
케이티텔보안네트워크에 대한 파일을 처리하고 저장했습니다.
케이티텔보안네트워크에 대한 이메일을 bough38@naver.com로 전송했습니다.
커넥션네트워크에 대한 파일을 처리하고 저장했습니다.
커넥션네트워크에 대한 이메일을 bough38@naver.com로 전송했습니다.
애드라인에 대한 파일을 처리하고 저장했습니다.
애드라인에 대한 이메일을 bough38@naver.com로 전송했습니다.
제제컴퍼니에 대한 파일을 처리하고 저장했습니다.
제제컴퍼니에 대한 이메일을 bough38@naver.com로 전송했습니다.
아이티인프라에 대한 파일을 처리하고 저장했습니다.
아이티인프라에 대한 이메일을 bough38@naver.com로 전송했습니다.
모든 파일이 처리되고 이메일이 전송되었습니다.


In [60]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 하나의 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")
    
    # 이메일 전송
    receiver_email = dealer_info[agency]
    
    # 이메일 메시지 설정
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"{agency} 4월 정산 파일 전송"
    
    # 이메일 본문
    body = f"""
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>첨부된 파일은 4월 정산 내역입니다. 확인 부탁드립니다.</p>
        <p>보내는 사람: 장재혁</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        <p><b>신규상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>
        {"<p><b>해지상세:</b></p>" + cancel_detail_html if not cancel_detail_filtered.empty else ""}
    </body>
    </html>
    """
    
    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/613639554.py:128: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()


NameError: name 'cancel_detail_html' is not defined

In [61]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 하나의 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
   
    print(f"{agency}에 대한 파일을 처리하고 저장했습니다.")
    
    # 이메일 전송
    receiver_email = dealer_info[agency]
    
    # 이메일 메시지 설정
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"{agency} 4월 정산 파일 전송"
    
    # 해지상세 HTML 생성
    cancel_detail_html = ""
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        """.format(
            ''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])
        )
    
    # 이메일 본문
    body = f"""
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>첨부된 파일은 4월 정산 내역입니다. 확인 부탁드립니다.</p>
        <p>보내는 사람: 장재혁</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        <p><b>신규상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """
    
    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/1684796621.py:128: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()


구정례에 대한 이메일을 bough38@naver.com로 전송했습니다.
케이티텔보안네트워크에 대한 파일을 처리하고 저장했습니다.
케이티텔보안네트워크에 대한 이메일을 bough38@naver.com로 전송했습니다.
커넥션네트워크에 대한 파일을 처리하고 저장했습니다.
커넥션네트워크에 대한 이메일을 bough38@naver.com로 전송했습니다.
애드라인에 대한 파일을 처리하고 저장했습니다.
애드라인에 대한 이메일을 bough38@naver.com로 전송했습니다.
제제컴퍼니에 대한 파일을 처리하고 저장했습니다.
제제컴퍼니에 대한 이메일을 bough38@naver.com로 전송했습니다.
아이티인프라에 대한 파일을 처리하고 저장했습니다.
아이티인프라에 대한 이메일을 bough38@naver.com로 전송했습니다.
모든 파일이 처리되고 이메일이 전송되었습니다.


In [62]:
pip install pandas openpyxl chardet


Note: you may need to restart the kernel to use updated packages.


In [64]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(agency, na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 하나의 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'X' or cell.column_letter == 'Y':  # 해지환수금액(별도세금계산서발행), 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['I1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['I1'].font = Font(bold=True)
            ws['I1'].number_format = '#,##0'
            ws['V1'] = new_detail_filtered['지급수수료'].sum()  # 지급수수료 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['I1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['I1'].font = Font(bold=True)
            ws['I1'].number_format = '#,##0'
            ws['V1'] = new_detail_filtered['지급수수료'].sum()  # 지급수수료 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['I1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['I1'].font = Font(bold=True)
            ws['I1'].number_format = '#,##0'
            ws['AB1'] = cancel_detail_filtered['환수금액'].sum()  # 환수금액 합계
            ws['AB1'].font = Font(bold=True, color='FF0000')
            ws['AB1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            f'{cancel_detail_filtered["환수금액"].sum():,}'
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].sum()

    # 이메일 본문
    body = f"""
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total:,}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/4126411428.py:144: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  summary_filtered = df_summary[df_summary['상호명'].str.contains(agency, na=False)].copy()


ValueError: Cannot specify ',' with 's'.

In [66]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 필터링된 데이터를 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            f'{cancel_detail_filtered["환수금액"].sum():,}'
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].sum()

    # 이메일 본문
    body = f"""
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total:,}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

ValueError: Cannot specify ',' with 's'.

In [68]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            f'{cancel_detail_filtered["환수금액"].sum():,}'
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].sum()

    # 이메일 본문
    body = """
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {summary_headers}
            </tr>
            {summary_rows}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total:,}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {new_detail_headers}
            </tr>
            {new_detail_rows}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """.format(
        agency=agency,
        summary_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns]),
        summary_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values]),
        new_detail_total=new_detail_total,
        new_detail_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns]),
        new_detail_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values]),
        cancel_detail_html=cancel_detail_html
    )

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

ValueError: Cannot specify ',' with 's'.

In [70]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            f'{cancel_detail_filtered["환수금액"].sum():,}'
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].sum()

    # 이메일 본문
    body = """
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {summary_headers}
            </tr>
            {summary_rows}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total:,}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {new_detail_headers}
            </tr>
            {new_detail_rows}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """.format(
        agency=agency,
        summary_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns]),
        summary_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values]),
        new_detail_total=f'{new_detail_total:,}',
        new_detail_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns]),
        new_detail_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values]),
        cancel_detail_html=cancel_detail_html
    )

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

ValueError: Cannot specify ',' with 's'.

In [71]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            "{:,}".format(cancel_detail_filtered["환수금액"].sum())
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].sum()

    # 이메일 본문
    body = """
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {summary_headers}
            </tr>
            {summary_rows}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {new_detail_headers}
            </tr>
            {new_detail_rows}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """.format(
        agency=agency,
        summary_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns]),
        summary_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values]),
        new_detail_total="{:,}".format(new_detail_total),
        new_detail_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns]),
        new_detail_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values]),
        cancel_detail_html=cancel_detail_html
    )

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

ValueError: Cannot specify ',' with 's'.

In [72]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            "{:,.0f}".format(cancel_detail_filtered["환수금액"].sum())
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].sum()

    # 이메일 본문
    body = """
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {summary_headers}
            </tr>
            {summary_rows}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {new_detail_headers}
            </tr>
            {new_detail_rows}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """.format(
        agency=agency,
        summary_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns]),
        summary_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values]),
        new_detail_total="{:,.0f}".format(new_detail_total),
        new_detail_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns]),
        new_detail_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values]),
        cancel_detail_html=cancel_detail_html
    )

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

ValueError: Unknown format code 'f' for object of type 'str'

In [73]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            "{:,.0f}".format(cancel_detail_filtered["환수금액"].sum())
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].sum()

    # 이메일 본문
    body = """
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {summary_headers}
            </tr>
            {summary_rows}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {new_detail_headers}
            </tr>
            {new_detail_rows}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """.format(
        agency=agency,
        summary_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns]),
        summary_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values]),
        new_detail_total="{:,.0f}".format(int(new_detail_total)),
        new_detail_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns]),
        new_detail_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values]),
        cancel_detail_html=cancel_detail_html
    )

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

ValueError: invalid literal for int() with base 10: '122,500875,000210,000122,500262,500175,000175,000238,000490,000175,000175,000210,000'

In [74]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            "{:,.0f}".format(cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum())
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = """
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {summary_headers}
            </tr>
            {summary_rows}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {new_detail_headers}
            </tr>
            {new_detail_rows}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """.format(
        agency=agency,
        summary_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns]),
        summary_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values]),
        new_detail_total="{:,.0f}".format(new_detail_total),
        new_detail_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns]),
        new_detail_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values]),
        cancel_detail_html=cancel_detail_html
    )

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

ValueError: could not convert string to float: ' - '

In [75]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월_정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            "{:,.0f}".format(cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum())
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = """
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {summary_headers}
            </tr>
            {summary_rows}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {new_detail_headers}
            </tr>
            {new_detail_rows}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """.format(
        agency=agency,
        summary_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns]),
        summary_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values]),
        new_detail_total="{:,.0f}".format(new_detail_total),
        new_detail_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns]),
        new_detail_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values]),
        cancel_detail_html=cancel_detail_html
    )

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [76]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span style="color: red; font-weight: bold; text-decoration: underline;">{2}</span></p>
        """.format(
            ''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            "{:,.0f}".format(cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum())
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = """
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        <p><b>표(요약):</b></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {summary_headers}
            </tr>
            {summary_rows}
        </table>
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold; text-decoration: underline;">지급수수료 합계: {new_detail_total}</span></p>
        <table border="1" style="border-collapse: collapse; font-size: 12px;">
            <tr>
                {new_detail_headers}
            </tr>
            {new_detail_rows}
        </table>
        {cancel_detail_html}
    </body>
    </html>
    """.format(
        agency=agency,
        summary_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in summary_filtered.columns]),
        summary_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values]),
        new_detail_total="{:,.0f}".format(new_detail_total),
        new_detail_headers=''.join([f'<th style="background-color: #000080; color: #FFFFFF;">{col}</th>' for col in new_detail_filtered.columns]),
        new_detail_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values]),
        cancel_detail_html=cancel_detail_html
    )

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [79]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)



Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [81]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = """
        <p><b>해지상세:</b></p>
        <table>
            <tr>
                {0}
            </tr>
            {1}
        </table>
        <p><b>환수금액 합계:</b> <span class="highlight-red">{2}</span></p>
        """.format(
            ''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns]),
            ''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values]),
            "{:,.0f}".format(cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum())
        )
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = """
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                font-size: 14px;
                line-height: 1.6;
            }
            p {
                margin: 0 0 15px;
            }
            table {
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
            }
            th, td {
                border: 1px solid #dddddd;
                text-align: left;
                padding: 8px;
            }
            th {
                background-color: #000080;
                color: white;
            }
            tr:nth-child(even) {
                background-color: #f2f2f2;
            }
            .highlight-blue {
                color: blue;
                font-weight: bold;
                text-decoration: underline;
            }
            .highlight-red {
                color: red;
                font-weight: bold;
                text-decoration: underline;
            }
            .summary-header {
                background-color: #000080;
                color: white;
            }
        </style>
    </head>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b></p>
        <table>
            <tr class="summary-header">
                {summary_headers}
            </tr>
            {summary_rows}
        </table>
        
        <p><b>신규상세:</b> <span class="highlight-blue">지급수수료 합계: {new_detail_total}</span></p>
        <table>
            <tr>
                {new_detail_headers}
            </tr>
            {new_detail_rows}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """.format(
        agency=agency,
        summary_headers=''.join([f'<th>{col}</th>' for col in summary_filtered.columns]),
        summary_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values]),
        new_detail_total="{:,.0f}".format(new_detail_total),
        new_detail_headers=''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns]),
        new_detail_rows=''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values]),
        cancel_detail_html=cancel_detail_html
    )

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

KeyError: '\n                font-family'

In [82]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000080", end_color="000080", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span class="highlight-red">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                font-size: 14px;
                line-height: 1.6;
            }}
            p {{
                margin: 0 0 15px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
            }}
            th, td {{
                border: 1px solid #dddddd;
                text-align: left;
                padding: 8px;
            }}
            th {{
                background-color: #000080;
                color: white;
            }}
            tr:nth-child(even) {{
                background-color: #f2f2f2;
            }}
            .highlight-blue {{
                color: blue;
                font-weight: bold;
                text-decoration: underline;
            }}
            .highlight-red {{
                color: red;
                font-weight: bold;
                text-decoration: underline;
            }}
            .summary-header {{
                background-color: #000080;
                color: white;
            }}
        </style>
    </head>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b></p>
        <table>
            <tr class="summary-header">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        
        <p><b>신규상세:</b> <span class="highlight-blue">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [83]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='DDDDDD'), 
                     right=Side(style='thin', color='DDDDDD'), 
                     top=Side(style='thin', color='DDDDDD'), 
                     bottom=Side(style='thin', color='DDDDDD'))
thick_border = Border(left=Side(style='thick', color='DDDDDD'), 
                      right=Side(style='thick', color='DDDDDD'), 
                      top=Side(style='thick', color='DDDDDD'), 
                      bottom=Side(style='thick', color='DDDDDD'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span class="highlight-red">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                font-size: 14px;
                line-height: 1.6;
            }}
            p {{
                margin: 0 0 15px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
            }}
            th, td {{
                border: 1px solid #dddddd;
                text-align: left;
                padding: 8px;
            }}
            th {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
            tr:nth-child(even) {{
                background-color: #f2f2f2;
            }}
            .highlight-blue {{
                color: blue;
                font-weight: bold;
                text-decoration: underline;
            }}
            .highlight-red {{
                color: red;
                font-weight: bold;
                text-decoration: underline;
            }}
            .summary-header {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b></p>
        <table>
            <tr class="summary-header">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        
        <p><b>신규상세:</b> <span class="highlight-blue">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [84]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span class="highlight-red">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                font-size: 14px;
                line-height: 1.6;
            }}
            p {{
                margin: 0 0 15px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
            }}
            th, td {{
                border: 1px solid #000000;
                text-align: left;
                padding: 8px;
            }}
            th {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
            tr:nth-child(even) {{
                background-color: #f2f2f2;
            }}
            .highlight-blue {{
                color: blue;
                font-weight: bold;
                text-decoration: underline;
            }}
            .highlight-red {{
                color: red;
                font-weight: bold;
                text-decoration: underline;
            }}
            .summary-header {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b></p>
        <table>
            <tr class="summary-header">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        
        <p><b>신규상세:</b> <span class="highlight-blue">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [85]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    if agency == "(주)미래넷":
        excel_output_path = os.path.join(output_base_path, "(주)미레넷_4월정산.xlsx")
    else:
        excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span class="highlight-red">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                font-size: 14px;
                line-height: 1.6;
            }}
            p {{
                margin: 0 0 15px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
                border: 2px dashed #000000;
            }}
            th, td {{
                border: 1px solid #000000;
                text-align: left;
                padding: 8px;
            }}
            th {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
            tr:nth-child(even) {{
                background-color: #f2f2f2;
            }}
            .highlight-blue {{
                color: blue;
                font-weight: bold;
                text-decoration: underline;
            }}
            .highlight-red {{
                color: red;
                font-weight: bold;
                text-decoration: underline;
            }}
            .summary-header {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b></p>
        <table>
            <tr class="summary-header">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        
        <p><b>신규상세:</b> <span class="highlight-blue">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [86]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    if agency == "(주)미래넷":
        excel_output_path = os.path.join(output_base_path, "(주)미레넷_4월정산.xlsx")
    else:
        excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span class="highlight-red">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                font-size: 14px;
                line-height: 1.6;
            }}
            p {{
                margin: 0 0 15px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
                border: 1px solid #A52A2A;
            }}
            th, td {{
                border: 1px solid #A52A2A;
                text-align: left;
                padding: 8px;
            }}
            th {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
            tr:nth-child(even) {{
                background-color: #f2f2f2;
            }}
            .highlight-blue {{
                color: blue;
                font-weight: bold;
                text-decoration: underline;
            }}
            .highlight-red {{
                color: red;
                font-weight: bold;
                text-decoration: underline;
            }}
            .summary-header {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b></p>
        <table>
            <tr class="summary-header">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        
        <p><b>신규상세:</b> <span class="highlight-blue">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [87]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    if agency == "(주)미래넷":
        excel_output_path = os.path.join(output_base_path, "(주)미레넷_4월정산.xlsx")
    else:
        excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span class="highlight-red">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                font-size: 14px;
                line-height: 1.6;
            }}
            p {{
                margin: 0 0 15px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
                background-color: #ffffff;
            }}
            th, td {{
                border: 1px solid #000000;
                text-align: left;
                padding: 8px;
            }}
            th {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
            tr:nth-child(even) {{
                background-color: #f2f2f2;
            }}
            .highlight-blue {{
                color: blue;
                font-weight: bold;
                text-decoration: underline;
            }}
            .highlight-red {{
                color: red;
                font-weight: bold;
                text-decoration: underline;
            }}
            .summary-header {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b></p>
        <table>
            <tr class="summary-header">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        
        <p><b>신규상세:</b> <span class="highlight-blue">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [88]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    if agency == "(주)미래넷":
        excel_output_path = os.path.join(output_base_path, "(주)미레넷_4월정산.xlsx")
    else:
        excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span class="highlight-red">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                font-size: 14px;
                line-height: 1.6;
            }}
            p {{
                margin: 0 0 15px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
                background-color: #ffffff;
            }}
            th, td {{
                border: 1px solid #000000;
                text-align: left;
                padding: 8px;
            }}
            th {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
            tr:nth-child(even) {{
                background-color: #f2f2f2;
            }}
            .highlight-blue {{
                color: blue;
                font-weight: bold;
                text-decoration: underline;
            }}
            .highlight-red {{
                color: red;
                font-weight: bold;
                text-decoration: underline;
            }}
            .summary-header {{
                background-color: #000000;
                color: white;
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b></p>
        <table>
            <tr class="summary-header">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        
        <p><b>신규상세:</b> <span class="highlight-blue">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table>
            <tr>
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [89]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    if agency == "(주)미래넷":
        excel_output_path = os.path.join(output_base_path, "(주)미레넷_4월정산.xlsx")
    else:
        excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table border="1" cellspacing="0" cellpadding="5">
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b></p>
        <table border="1" cellspacing="0" cellpadding="5">
            <tr style="background-color: #000000; color: #FFFFFF; font-weight: bold;">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table border="1" cellspacing="0" cellpadding="5">
            <tr style="background-color: #000000; color: #FFFFFF; font-weight: bold;">
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [90]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '본부', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '프로모션', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '비고', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# 열 이름을 맞추기 위해 필요한 열만 남기고 순서대로 정렬
df_summary = df_summary[required_columns]

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=["Unnamed: 39"], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    if agency == "(주)미래넷":
        excel_output_path = os.path.join(output_base_path, "(주)미레넷_4월정산.xlsx")
    else:
        excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table border="1" cellspacing="0" cellpadding="5">
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b> 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}, 최종지급액: {summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}, 해지환수금액: {summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}, 승인번호: {', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</p>
        <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
            <tr style="background-color: #000000; color: #FFFFFF; font-weight: bold;">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns if col not in ['본부', '프로모션', '비고']])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered[summary_filtered.columns.difference(['본부', '프로모션', '비고'])].values])}
        </table>
        
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
            <tr style="background-color: #000000; color: #FFFFFF; font-weight: bold;">
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns if col not in ['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고']])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered[new_detail_filtered.columns.difference(['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'])].values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [98]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 필요한 열 제외
df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "bough38@naver.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    if agency == "(주)미래넷":
        excel_output_path = os.path.join(output_base_path, "(주)미레넷_4월정산.xlsx")
    else:
        excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
        # 특정 컬럼 스타일 적용
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = thin_border
                if cell.column_letter == 'V':  # 최종지급액 컬럼
                    cell.font = blue_font
                elif cell.column_letter == 'Z':  # 해지환수금액(별도세금계산서발행) 컬럼
                    cell.font = red_font
                elif cell.column_letter == 'AA':  # 승인번호 컬럼
                    cell.font = red_font

        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['V1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'
            ws['Z1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['Z1'].font = Font(bold=True, color='FF0000')
            ws['Z1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['R1'].font = Font(bold=True, color='0000FF')
            ws['R1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['R1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['R1'].font = Font(bold=True, color='FF0000')
            ws['R1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table border="1" cellspacing="0" cellpadding="5">
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        <p><b>표(요약):</b> 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}, 최종지급액: {summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}, 해지환수금액: {summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}, 승인번호: {', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</p>
        <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
            <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
            <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [116]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 필요한 열 제외
df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "heebon.park@kt.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
sender_email = "bough38@naver.com"
sender_password = "Plus3867@@"  # 실제 비밀번호를 입력하세요
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

    # 해지상세 데이터 필터링
    cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
    cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

    # 표(요약) 승인번호 값이 없으면 해지상세 제외
    if summary_filtered['승인번호'].isnull().all():
        cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

    # ' - ' 문자를 0으로 대체
    summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
    
    # 엑셀 파일로 저장
    if agency == "(주)미래넷":
        excel_output_path = os.path.join(output_base_path, "(주)미래넷_4월정산.xlsx")
    else:
        excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
        new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
        if not cancel_detail_filtered.empty:
            cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
    
    # 엑셀 파일 스타일링
    wb = openpyxl.load_workbook(excel_output_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        
        # 헤더 스타일 적용
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.border = thick_border
        
       
        # 합계 추가
        if sheet == "표(요약)":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()  # 최종지급액 합계
            ws['U1'].font = Font(bold=True, color='0000FF')
            ws['U1'].number_format = '#,##0'
            ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()  # 해지환수금액 합계
            ws['V1'].font = Font(bold=True, color='FF0000')
            ws['V1'].number_format = '#,##0'

        if sheet == "신규상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['H1'] = new_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['H1'].font = Font(bold=True)
            ws['H1'].number_format = '#,##0'
            ws['V1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()  # 지급수수료 합계
            ws['V1'].font = Font(bold=True, color='0000FF')
            ws['V1'].number_format = '#,##0'

        if sheet == "해지상세":
            ws.insert_rows(1)
            ws['A1'] = '합계'
            ws['A1'].font = Font(bold=True)
            ws['E1'] = cancel_detail_filtered['계약번호'].nunique()  # 계약번호 건수 합계
            ws['E1'].font = Font(bold=True)
            ws['E1'].number_format = '#,##0'
            ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()  # 환수금액 합계
            ws['AF1'].font = Font(bold=True, color='FF0000')
            ws['AF1'].number_format = '#,##0'
    
    wb.save(excel_output_path)

    # 이메일 전송
    receiver_email = dealer_info[agency]
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

    # 해지상세 HTML 생성
    if not cancel_detail_filtered.empty:
        cancel_detail_html = f"""
        <p><b>해지상세:</b></p>
        <table border="1" cellspacing="0" cellpadding="5">
            <tr>
                {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
        </table>
        <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
        """
    else:
        cancel_detail_html = ""

    # 신규상세 합계 계산
    new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

    # 이메일 본문
    body = f"""
    <!DOCTYPE html>
    <html>
    <body>
        <p>{agency}님, 안녕하세요.</p>
        <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
        <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
        
        </p>
        
        <p><b>표(요약):<p><b> ㅇ 최종지급액 : <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span>
        
        </p>
         <p><b> ㅇ 해지환수금액(세금계산서별도발행) : <span style="color: red; font-weight: bold;">{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span> 
        </p>
        <p><b><b>  ㅇ 해지 전자계산서 승인번호: <span style="color: green; font-weight: bold;">{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>
        </p>
        <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
            <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
        </table>
        

        
        
        <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_total:,.0f}</span></p>
        <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
            <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
            </tr>
            {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
        </table>

        {cancel_detail_html}
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    # 파일 첨부
    with open(excel_output_path, "rb") as attachment:
        part = MIMEApplication(attachment.read(), _subtype="octet-stream")
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
        msg.attach(part)

    # 이메일 전송
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

    print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

print("모든 파일이 처리되고 이메일이 전송되었습니다.")


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

In [117]:
pip install pyinstaller

  Obtaining dependency information for pyinstaller from https://files.pythonhosted.org/packages/19/29/d24aa7abe3cb9a1386130b073c4136546c4bc0da7c96bba1621f6198bfc9/pyinstaller-6.6.0-py3-none-macosx_10_13_universal2.whl.metadata
  Obtaining dependency information for altgraph from https://files.pythonhosted.org/packages/4d/3f/3bc3f1d83f6e4a7fcb834d3720544ca597590425be5ba9db032b2bf322a2/altgraph-0.17.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyinstaller-hooks-contrib>=2024.3 from https://files.pythonhosted.org/packages/2e/84/079d3bd411f591d825dca78b83093fcc9631ea2f887cb01fa9e19f4c619f/pyinstaller_hooks_contrib-2024.6-py2.py3-none-any.whl.metadata
  Obtaining dependency information for macholib>=1.8 from https://files.pythonhosted.org/packages/d1/5d/c059c180c84f7962db0aeae7c3b9303ed1d73d76f2bfbc32bc231c8be314/macholib-1.16.3-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 959.8/959.8 kB 6.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━

In [118]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox

# GUI 창을 생성
root = tk.Tk()
root.withdraw()  # 메인 윈도우 숨김

# 사용자 입력 받기
sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

# 파일 읽기 (7행을 헤더로 설정하고, 6행까지 불필요한 행 건너뜀)
df_summary = read_file_with_encoding(summary_path, header=6)
df_new_detail = read_file_with_encoding(new_detail_path)
df_cancel_detail = read_file_with_encoding(cancel_detail_path)

# 데이터 확인
print("Summary DataFrame (Initial):")
print(df_summary.head())

# 'Unnamed' 열 제거
df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]

# 열 이름을 정규화
df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

# 필요한 열 이름 업데이트
required_columns = [
    '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
    '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
    '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
]

# 열 수가 맞지 않는 경우 필요한 열 추가
if len(df_summary.columns) < len(required_columns):
    for col in required_columns:
        if col not in df_summary.columns:
            df_summary[col] = None

# NaN 값을 공백으로 대체
df_summary = df_summary.fillna('')
df_new_detail = df_new_detail.fillna('')
df_cancel_detail = df_cancel_detail.fillna('')

# 데이터 확인 (열 수 확인)
print("Summary DataFrame columns after removing 'Unnamed':")
print(df_summary.columns)
print("Number of columns:", len(df_summary.columns))

# 7행에 값이 없는 경우 5행 또는 6행에서 값을 가져옴
def fill_missing_values(df, target_row, rows_to_check, cols):
    for col in cols:
        for row in rows_to_check:
            if pd.isna(df.at[target_row, col]):
                if not pd.isna(df.at[row, col]):
                    df.at[target_row, col] = df.at[row, col]
                    break

# 필요한 열 이름을 지정
columns_to_copy = df_summary.columns

# 7행에 결측치 채우기 (5행, 6행을 체크하여 7행에 결측치 채우기)
fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

# "강북/강원본부"에 해당하는 행만 남기고 "충남", "강남", "대구", "부산"으로 시작하는 본부 제외
df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]

# 필요한 열 제외
df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

# 총계행과 바로 아래 행 제외
total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
if not total_row_index.empty:
    df_summary = df_summary.drop(total_row_index)
    df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

# 데이터 확인 (필터링 후)
print("Summary DataFrame after filtering:")
print(df_summary.head())

# '서비스번호', '고객번호', '계약번호'의 소수점 제거
df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

# 대리점 목록
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

# 대리점 이메일 정보 설정
dealer_info = {
    "(주)미래넷": "heebon.park@kt.com",
    "구정례": "heebon.park@kt.com",
    "케이티텔보안네트워크": "bough38@naver.com",
    "커넥션네트워크": "bough38@naver.com",
    "애드라인": "bough38@naver.com",
    "제제컴퍼니": "bough38@naver.com",
    "아이티인프라": "bough38@naver.com"
}

# 이메일 설정
smtp_server = "smtp.naver.com"
smtp_port = 587

# 스타일 설정
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True)
blue_font = Font(color="0000FF", bold=True)
red_font = Font(color="FF0000", bold=True)
thin_border = Border(left=Side(style='thin', color='000000'), 
                     right=Side(style='thin', color='000000'), 
                     top=Side(style='thin', color='000000'), 
                     bottom=Side(style='thin', color='000000'))
thick_border = Border(left=Side(style='thick', color='000000'), 
                      right=Side(style='thick', color='000000'), 
                      top=Side(style='thick', color='000000'), 
                      bottom=Side(style='thick', color='000000'))

# 각 대리점에 대해 데이터 처리 및 별도 파일 저장
for agency in agencies:
    # 요약 데이터 필터링
    summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
    
    if summary_filtered.empty:
        print(f"{agency}에 대한 데이터가 없습니다.")
        continue

    summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)  # 중복된 행 제거
    
    # 신규상세 데이터 필터링
    new_detail


2024-05-22 07:25:10.210 python[91780:1743999] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


Summary DataFrame (Initial):
  Unnamed: 0 정산년월\n(YYMM)       본부      지사     추천자사번  대표자      상호명  \
0        NaN         2404  대구/경북본부   동대구지사  318715.0  배정영     수아정보   
1        NaN         2404  충남/충북본부  충남서부지사  317730.0  장주명   조양문텔레콤   
2        NaN         2404  강북/강원본부    고양지사  317856.0  전연진   (주)미래넷   
3        NaN         2404  부산/경남본부    울산지사  319419.0  이호준  서진(대리점)   
4        NaN         2404  대구/경북본부   동대구지사  317684.0  김유식  주식회사 나모   

        사업자등록번호      신규월정료 신규건수  ...    판매형  소급액 전월미지급액 전월미환수        공급가액  \
0  3.107600e+09    179,000    3  ...     -   NaN    NaN   NaN     391,800   
1  3.100278e+09    400,450    4  ...     -   NaN    NaN   NaN     955,000   
2  1.288647e+09    923,000   12  ...     -   NaN    NaN   NaN   4,271,790   
3  5.911602e+09  3,874,000   60  ...     -   NaN    NaN   NaN  18,595,200   
4  5.038172e+09    672,000   15  ...  3,000  NaN    NaN   NaN   1,935,610   

  부가세\n(공급가액10%)       최종지급액   비고 해지환수금액\n(별도세금계산서발행) 승인번호  
0         39,180     430,9

NameError: name 'new_detail' is not defined

In [119]:
pip install pyinstaller

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pyinstaller --onefile --windowed your_script_name.py

In [122]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("대리점 및 이메일 설정")
        self.dealer_info = {
            "(주)미래넷": "heebon.park@kt.com",
            "구정례": "heebon.park@kt.com",
            "케이티텔보안네트워크": "bough38@naver.com",
            "커넥션네트워크": "bough38@naver.com",
            "애드라인": "bough38@naver.com",
            "제제컴퍼니": "bough38@naver.com",
            "아이티인프라": "bough38@naver.com"
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings')
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def main():
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()

    sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
    sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')

    df_summary = read_file_with_encoding(summary_path, header=6)
    df_new_detail = read_file_with_encoding(new_detail_path)
    df_cancel_detail = read_file_with_encoding(cancel_detail_path)

    df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
    df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

    required_columns = [
        '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
        '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
        '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
    ]

    if len(df_summary.columns) < len(required_columns):
        for col in required_columns:
            if col not in df_summary.columns:
                df_summary[col] = None

    df_summary = df_summary.fillna('')
    df_new_detail = df_new_detail.fillna('')
    df_cancel_detail = df_cancel_detail.fillna('')

    def fill_missing_values(df, target_row, rows_to_check, cols):
        for col in cols:
            for row in rows_to_check:
                if pd.isna(df.at[target_row, col]):
                    if not pd.isna(df.at[row, col]):
                        df.at[target_row, col] = df.at[row, col]
                        break

    columns_to_copy = df_summary.columns
    fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

    df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
    df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
    df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

    total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
    if not total_row_index.empty:
        df_summary = df_summary.drop(total_row_index)
        df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

    df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
    df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

    df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
   
    df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

    agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

    dealer_info = {
        "(주)미래넷": "heebon.park@kt.com",
        "구정례": "heebon.park@kt.com",
        "케이티텔보안네트워크": "bough38@naver.com",
        "커넥션네트워크": "bough38@naver.com",
        "애드라인": "bough38@naver.com",
        "제제컴퍼니": "bough38@naver.com",
        "아이티인프라": "bough38@naver.com"
    }

    smtp_server = "smtp.naver.com"
    smtp_port = 587

    header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    blue_font = Font(color="0000FF", bold=True)
    red_font = Font(color="FF0000", bold=True)
    thin_border = Border(left=Side(style='thin', color='000000'), 
                         right=Side(style='thin', color='000000'), 
                         top=Side(style='thin', color='000000'), 
                         bottom=Side(style='thin', color='000000'))
    thick_border = Border(left=Side(style='thick', color='000000'), 
                          right=Side(style='thick', color='000000'), 
                          top=Side(style='thick', color='000000'), 
                          bottom=Side(style='thick', color='000000'))

    for agency in agencies:
        summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
        
        if summary_filtered.empty:
            print(f"{agency}에 대한 데이터가 없습니다.")
            continue

        summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
        
        new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

        cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

        if summary_filtered['승인번호'].isnull().all():
            cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

        summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
        
        if agency == "(주)미래넷":
            excel_output_path = os.path.join(output_base_path, "(주)미래넷_4월정산.xlsx")
        else:
            excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

        with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
            summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
            new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
            if not cancel_detail_filtered.empty:
                cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
        
        wb = openpyxl.load_workbook(excel_output_path)
        
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thick_border
            
            if sheet == "표(요약)":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['U1'].font = Font(bold=True, color='0000FF')
                ws['U1'].number_format = '#,##0'
                ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='FF0000')
                ws['V1'].number_format = '#,##0'

            if sheet == "신규상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['H1'] = new_detail_filtered['계약번호'].nunique()
                ws['H1'].font = Font(bold=True)
                ws['H1'].number_format = '#,##0'
                ws['V1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='0000FF')
                ws['V1'].number_format = '#,##0'

            if sheet == "해지상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['E1'] = cancel_detail_filtered['계약번호'].nunique()
                ws['E1'].font = Font(bold=True)
                ws['E1'].number_format = '#,##0'
                ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AF1'].font = Font(bold=True, color='FF0000')
                ws['AF1'].number_format = '#,##0'
        
        wb.save(excel_output_path)

        receiver_email = dealer_info[agency]
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

        if not cancel_detail_filtered.empty:
            cancel_detail_html = f"""
            <p><b>해지상세:</b></p>
            <table border="1" cellspacing="0" cellpadding="5">
                <tr>
                    {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
            </table>
            <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            """
        else:
            cancel_detail_html = ""

        new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

        body = f"""
        <!DOCTYPE html>
        <html>
        <body>
            <p>{agency}님, 안녕하세요.</p>
            <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
            <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
            
            <p><b>표(요약):</b> 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}, 최종지급액: <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span>, 해지환수금액: <span style="color: red; font-weight: bold;">{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span>, 승인번호: <span style="color: green; font-weight: bold;">{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>
            <a href="javascript:void(0);" onclick="document.getElementById('summary_table').style.display='block';">내역 보기</a>
            <div id="summary_table" style="display:none;">
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
                </table>
            </div>
            
            <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_total:,.0f}</span></p>
            <a href="javascript:void(0);" onclick="document.getElementById('new_detail_table').style.display='block';">내역 보기</a>
            <div id="new_detail_table" style="display:none;">
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
                </table>
            </div>

            {cancel_detail_html}
        </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        with open(excel_output_path, "rb") as attachment:
            part = MIMEApplication(attachment.read(), _subtype="octet-stream")
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
            msg.attach(part)

        # 이메일 전송
        def send_email():
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()
                server.login(sender_email, sender_password)
                server.sendmail(sender_email, receiver_email, msg.as_string())
            print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

        # 미리보기 창
        def preview_email():
            preview = tk.Tk()
            preview.title("이메일 미리보기")

            preview_frame = tk.Frame(preview)
            preview_frame.pack(pady=10)

            email_text = tk.Text(preview_frame, wrap='word', width=100, height=40)
            email_text.insert('1.0', body)
            email_text.pack(side=tk.LEFT)

            scrollbar = tk.Scrollbar(preview_frame, orient=tk.VERTICAL, command=email_text.yview)
            email_text.configure(yscroll=scrollbar.set)
            scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

            button_frame = tk.Frame(preview)
            button_frame.pack(pady=10)
            tk.Button(button_frame, text="전송", command=lambda: [send_email(), preview.destroy()]).pack(side=tk.LEFT, padx=5)
            tk.Button(button_frame, text="취소", command=preview.destroy).pack(side=tk.LEFT, padx=5)

            preview.mainloop()

        preview_email()

    print("모든 파일이 처리되고 이메일이 전송되었습니다.")

if __name__ == "__main__":
    main()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/heebonpark/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/4067822642.py", line 378, in <lambda>
    tk.Button(button_frame, text="전송", command=lambda: [send_email(), preview.destroy()]).pack(side=tk.LEFT, padx=5)
                                                        ^^^^^^^^^^^^
  File "/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/4067822642.py", line 356, in send_email
    server.login(sender_email, sender_password)
  File "/Users/heebonpark/anaconda3/lib/python3.11/smtplib.py", line 750, in login
    raise last_exception
  File "/Users/heebonpark/anaconda3/lib/python3.11/smtplib.py", line 739, in login
    (code, resp) = self.auth(
                   ^^^^^^^^^^
  File "/Users/heebonpark/anaconda3/lib/python3.11/smtplib.py", line 662, 

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/heebonpark/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/4067822642.py", line 378, in <lambda>
    tk.Button(button_frame, text="전송", command=lambda: [send_email(), preview.destroy()]).pack(side=tk.LEFT, padx=5)
                                                        ^^^^^^^^^^^^
  File "/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/4067822642.py", line 356, in send_email
    server.login(sender_email, sender_password)
  File "/Users/heebonpark/anaconda3/lib/python3.11/smtplib.py", line 750, in login
    raise last_exception
  File "/Users/heebonpark/anaconda3/lib/python3.11/smtplib.py", line 739, in login
    (code, resp) = self.auth(
                   ^^^^^^^^^^
  File "/Users/heebonpark/anaconda3/lib/python3.11/smtplib.py", line 662, 

KeyboardInterrupt: 

In [128]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk, scrolledtext

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("대리점 및 이메일 설정")
        self.dealer_info = {
            "(주)미래넷": "heebon.park@kt.com",
            "구정례": "heebon.park@kt.com",
            "케이티텔보안네트워크": "bough38@naver.com",
            "커넥션네트워크": "bough38@naver.com",
            "애드라인": "bough38@naver.com",
            "제제컴퍼니": "bough38@naver.com",
            "아이티인프라": "bough38@naver.com"
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings')
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def main():
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()

    sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
    sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')

    df_summary = read_file_with_encoding(summary_path, header=6)
    df_new_detail = read_file_with_encoding(new_detail_path)
    df_cancel_detail = read_file_with_encoding(cancel_detail_path)

    df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
    df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

    required_columns = [
        '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
        '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
        '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
    ]

    if len(df_summary.columns) < len(required_columns):
        for col in required_columns:
            if col not in df_summary.columns:
                df_summary[col] = None

    df_summary = df_summary.fillna('')
    df_new_detail = df_new_detail.fillna('')
    df_cancel_detail = df_cancel_detail.fillna('')

    def fill_missing_values(df, target_row, rows_to_check, cols):
        for col in cols:
            for row in rows_to_check:
                if pd.isna(df.at[target_row, col]):
                    if not pd.isna(df.at[row, col]):
                        df.at[target_row, col] = df.at[row, col]
                        break

    columns_to_copy = df_summary.columns
    fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

    df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
    df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
    df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

    total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
    if not total_row_index.empty:
        df_summary = df_summary.drop(total_row_index)
        df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

    df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
    df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

    df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]
                                                                                       
    agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

    smtp_server = "smtp.naver.com"
    smtp_port = 587

    header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    blue_font = Font(color="0000FF", bold=True)
    red_font = Font(color="FF0000", bold=True)
    thin_border = Border(left=Side(style='thin', color='000000'), 
                         right=Side(style='thin', color='000000'), 
                         top=Side(style='thin', color='000000'), 
                         bottom=Side(style='thin', color='000000'))
    thick_border = Border(left=Side(style='thick', color='000000'), 
                          right=Side(style='thick', color='000000'), 
                          top=Side(style='thick', color='000000'), 
                          bottom=Side(style='thick', color='000000'))

    for agency in agencies:
        summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
        
        if summary_filtered.empty:
            print(f"{agency}에 대한 데이터가 없습니다.")
            continue

        summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
        
        new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

        cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

        if summary_filtered['승인번호'].isnull().all():
            cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

        summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
        
        if agency == "(주)미래넷":
            excel_output_path = os.path.join(output_base_path, "(주)미래넷_4월정산.xlsx")
        else:
            excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

        with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
            summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
            new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
            if not cancel_detail_filtered.empty:
                cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
        
        wb = openpyxl.load_workbook(excel_output_path)
        
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thick_border
            
            if sheet == "표(요약)":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['U1'].font = Font(bold=True, color='0000FF')
                ws['U1'].number_format = '#,##0'
                ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='FF0000')
                ws['V1'].number_format = '#,##0'

            if sheet == "신규상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['H1'] = new_detail_filtered['계약번호'].nunique()
                ws['H1'].font = Font(bold=True)
                ws['H1'].number_format = '#,##0'
                ws['V1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='0000FF')
                ws['V1'].number_format = '#,##0'

            if sheet == "해지상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['E1'] = cancel_detail_filtered['계약번호'].nunique()
                ws['E1'].font = Font(bold=True)
                ws['E1'].number_format = '#,##0'
                ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AF1'].font = Font(bold=True, color='FF0000')
                ws['AF1'].number_format = '#,##0'
        
        wb.save(excel_output_path)

        receiver_email = app.dealer_info[agency]
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

        if not cancel_detail_filtered.empty:
            cancel_detail_html = f"""
            <p><b>해지상세:</b></p>
            <table border="1" cellspacing="0" cellpadding="5">
                <tr>
                    {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
            </table>
            <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            """
        else:
            cancel_detail_html = ""

        new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

        body = f"""
        <!DOCTYPE html>
        <html>
        <body>
            <p>{agency}님, 안녕하세요.</p>
            <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
            <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
            
            <p><b>표(요약):</b> 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}, 최종지급액: <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span>, 해지환수금액: <span style="color: red; font-weight: bold;">{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span>, 승인번호: <span style="color: green; font-weight: bold;">{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>
            <a href="javascript:void(0);" onclick="document.getElementById('summary_table').style.display='block';">내역 보기</a>
            <div id="summary_table" style="display:none;">
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
                </table>
            </div>
            
            <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">
            
                        지급수수료 합계: {new_detail_total:,.0f}</span></p>
            <a href="javascript:void(0);" onclick="document.getElementById('new_detail_table').style.display='block';">내역 보기</a>
            <div id="new_detail_table" style="display:none;">
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
                </table>
            </div>

            {cancel_detail_html}
        </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        with open(excel_output_path, "rb") as attachment:
            part = MIMEApplication(attachment.read(), _subtype="octet-stream")
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
            msg.attach(part)

        # 이메일 전송
        def send_email():
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()
                server.login(sender_email, sender_password)
                server.sendmail(sender_email, receiver_email, msg.as_string())
            print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")

        # 미리보기 창
        def preview_email():
            preview = tk.Tk()
            preview.title("이메일 미리보기")

            preview_frame = tk.Frame(preview)
            preview_frame.pack(pady=10)

            email_text = scrolledtext.ScrolledText(preview_frame, wrap='word', width=100, height=40)
            email_text.insert(tk.END, body)
            email_text.pack(side=tk.LEFT)

            button_frame = tk.Frame(preview)
            button_frame.pack(pady=10)
            tk.Button(button_frame, text="전송", command=lambda: [send_email(), preview.destroy()]).pack(side=tk.LEFT, padx=5)
            tk.Button(button_frame, text="취소", command=preview.destroy).pack(side=tk.LEFT, padx=5)

            preview.mainloop()

        preview_email()

    print("모든 파일이 처리되고 이메일이 전송되었습니다.")

if __name__ == "__main__":
    main()




Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/heebonpark/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/3611803446.py", line 365, in <lambda>
    tk.Button(button_frame, text="전송", command=lambda: [send_email(), preview.destroy()]).pack(side=tk.LEFT, padx=5)
                                                        ^^^^^^^^^^^^
  File "/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_91780/3611803446.py", line 347, in send_email
    server.login(sender_email, sender_password)
  File "/Users/heebonpark/anaconda3/lib/python3.11/smtplib.py", line 750, in login
    raise last_exception
  File "/Users/heebonpark/anaconda3/lib/python3.11/smtplib.py", line 739, in login
    (code, resp) = self.auth(
                   ^^^^^^^^^^
  File "/Users/heebonpark/anaconda3/lib/python3.11/smtplib.py", line 662, 

KeyboardInterrupt: 

In [ ]:
./your_script_name

In [2]:
pip install pyinstaller

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("대리점 및 이메일 설정")
        self.dealer_info = {
            "(주)미래넷": "heebon.park@kt.com",
            "구정례": "heebon.park@kt.com",
            "케이티텔보안네트워크": "bough38@naver.com",
            "커넥션네트워크": "bough38@naver.com",
            "애드라인": "bough38@naver.com",
            "제제컴퍼니": "bough38@naver.com",
            "아이티인프라": "bough38@naver.com"
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings')
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def main():
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()

    # sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
    # sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')
    
     sender_email = simpledialog.askstring("bough38@naver.com")
    sender_password = simpledialog.askstring("Plus3867@@")

    df_summary = read_file_with_encoding(summary_path, header=6)
    df_new_detail = read_file_with_encoding(new_detail_path)
    df_cancel_detail = read_file_with_encoding(cancel_detail_path)

    df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
    df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

    required_columns = [
        '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
        '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
        '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
    ]

    if len(df_summary.columns) < len(required_columns):
        for col in required_columns:
            if col not in df_summary.columns:
                df_summary[col] = None

    df_summary = df_summary.fillna('')
    df_new_detail = df_new_detail.fillna('')
    df_cancel_detail = df_cancel_detail.fillna('')

    def fill_missing_values(df, target_row, rows_to_check, cols):
        for col in cols:
            for row in rows_to_check:
                if pd.isna(df.at[target_row, col]):
                    if not pd.isna(df.at[row, col]):
                        df.at[target_row, col] = df.at[row, col]
                        break

    columns_to_copy = df_summary.columns
    fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

    df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
    df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
    df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

    total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
    if not total_row_index.empty:
        df_summary = df_summary.drop(total_row_index)
        df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

    df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
    df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

    agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

    dealer_info = {
        "(주)미래넷": "heebon.park@kt.com",
        "구정례": "heebon.park@kt.com",
        "케이티텔보안네트워크": "bough38@naver.com",
        "커넥션네트워크": "bough38@naver.com",
        "애드라인": "bough38@naver.com",
        "제제컴퍼니": "bough38@naver.com",
        "아이티인프라": "bough38@naver.com"
    }

    smtp_server = "smtp.naver.com"
    smtp_port = 587

    header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    blue_font = Font(color="0000FF", bold=True)
    red_font = Font(color="FF0000", bold=True)
    thin_border = Border(left=Side(style='thin', color='000000'), 
                         right=Side(style='thin', color='000000'), 
                         top=Side(style='thin', color='000000'), 
                         bottom=Side(style='thin', color='000000'))
    thick_border = Border(left=Side(style='thick', color='000000'), 
                          right=Side(style='thick', color='000000'), 
                          top=Side(style='thick', color='000000'), 
                          bottom=Side(style='thick', color='000000'))

    for agency in agencies:
        summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
        
        if summary_filtered.empty:
            print(f"{agency}에 대한 데이터가 없습니다.")
            continue

        summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
        
        new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

        cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

        if summary_filtered['승인번호'].isnull().all():
            cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

        summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
        
        if agency == "(주)미래넷":
            excel_output_path = os.path.join(output_base_path, "(주)미래넷_4월정산.xlsx")
        else:
            excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

        with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
            summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
            new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
            if not cancel_detail_filtered.empty:
                cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
        
        wb = openpyxl.load_workbook(excel_output_path)
        
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thick_border
            
            if sheet == "표(요약)":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['U1'].font = Font(bold=True, color='0000FF')
                ws['U1'].number_format = '#,##0'
                ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='FF0000')
                ws['V1'].number_format = '#,##0'

            if sheet == "신규상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['H1'] = new_detail_filtered['계약번호'].nunique()
                ws['H1'].font = Font(bold=True)
                ws['H1'].number_format = '#,##0'
                ws['V1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='0000FF')
                ws['V1'].number_format = '#,##0'

            if sheet == "해지상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['E1'] = cancel_detail_filtered['계약번호'].nunique()
                ws['E1'].font = Font(bold=True)
                ws['E1'].number_format = '#,##0'
                ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AF1'].font = Font(bold=True, color='FF0000')
                ws['AF1'].number_format = '#,##0'
        
        wb.save(excel_output_path)

        receiver_email = dealer_info[agency]
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

        if not cancel_detail_filtered.empty:
            cancel_detail_html = f"""
            <p><b>해지상세:</b></p>
            <table border="1" cellspacing="0" cellpadding="5">
                <tr>
                    {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
            </table>
            <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            """
        else:
            cancel_detail_html = ""

        new_detail_total = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()

        body = f"""
        <!DOCTYPE html>
        <html>
        <body>
            <p>{agency}님, 안녕하세요.</p>
            <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
            <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
            
            <p><b>표(요약):</b> 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}, 최종지급액: <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span>, 해지환수금액: <span style="color: red; font-weight: bold;">{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span>, 승인번호
: <span style="color: green; font-weight: bold;">{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>
            <a href="javascript:void(0);" onclick="document.getElementById('summary_table').style.display='block';">내역 보기</a>
            <div id="summary_table" style="display:none;">
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
                </table>
            </div>
            
            <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_total:,.0f}</span></p>
            <a href="javascript:void(0);" onclick="document.getElementById('new_detail_table').style.display='block';">내역 보기</a>
            <div id="new_detail_table" style="display:none;">
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
                </table>
            </div>

            {cancel_detail_html}
        </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        with open(excel_output_path, "rb") as attachment:
            part = MIMEApplication(attachment.read(), _subtype="octet-stream")
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
            msg.attach(part)

        # 이메일 전송
        def send_email():
            try:
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()
                    server.login(sender_email, sender_password)
                    server.sendmail(sender_email, receiver_email, msg.as_string())
                messagebox.showinfo("성공", f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")
            except smtplib.SMTPAuthenticationError:
                messagebox.showerror("오류", "이메일 로그인에 실패했습니다. 이메일 주소와 비밀번호를 확인하세요.")

        # 미리보기 창
        def preview_email():
            preview = tk.Tk()
            preview.title("이메일 미리보기")

            preview_frame = tk.Frame(preview)
            preview_frame.pack(pady=10)

            email_text = tk.Text(preview_frame, wrap='word', width=100, height=40)
            email_text.insert('1.0', body)
            email_text.pack(side=tk.LEFT)

            scrollbar = tk.Scrollbar(preview_frame, orient=tk.VERTICAL, command=email_text.yview)
            email_text.configure(yscroll=scrollbar.set)
            scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

            button_frame = tk.Frame(preview)
            button_frame.pack(pady=10)
            tk.Button(button_frame, text="전송", command=lambda: [send_email(), preview.destroy()]).pack(side=tk.LEFT, padx=5)
            tk.Button(button_frame, text="취소", command=preview.destroy).pack(side=tk.LEFT, padx=5)

            preview.mainloop()

        preview_email()

    print("모든 파일이 처리되고 이메일이 전송되었습니다.")

if __name__ == "__main__":
    main()

    
    
    
    

IndentationError: unexpected indent (2908323972.py, line 133)

In [1]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("대리점 및 이메일 설정")
        self.dealer_info = {
            "(주)미래넷": "heebon.park@kt.com",
            "구정례": "heebon.park@kt.com",
            "케이티텔보안네트워크": "bough38@naver.com",
            "커넥션네트워크": "bough38@naver.com",
            "애드라인": "bough38@naver.com",
            "제제컴퍼니": "bough38@naver.com",
            "아이티인프라": "bough38@naver.com"
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings')
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def main():
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()

    sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
    sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')

    df_summary = read_file_with_encoding(summary_path, header=6)
    df_new_detail = read_file_with_encoding(new_detail_path)
    df_cancel_detail = read_file_with_encoding(cancel_detail_path)

    df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
    df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

    required_columns = [
        '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
        '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
        '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
    ]

    if len(df_summary.columns) < len(required_columns):
        for col in required_columns:
            if col not in df_summary.columns:
                df_summary[col] = None

    df_summary = df_summary.fillna('')
    df_new_detail = df_new_detail.fillna('')
    df_cancel_detail = df_cancel_detail.fillna('')

    def fill_missing_values(df, target_row, rows_to_check, cols):
        for col in cols:
            for row in rows_to_check:
                if pd.isna(df.at[target_row, col]):
                    if not pd.isna(df.at[row, col]):
                        df.at[target_row, col] = df.at[row, col]
                        break

    columns_to_copy = df_summary.columns
    fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

    df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
    df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
    df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

    total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
    if not total_row_index.empty:
        df_summary = df_summary.drop(total_row_index)
        df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

    df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
    df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

    df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]
    agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

    dealer_info = {
        "(주)미래넷": "heebon.park@kt.com",
        "구정례": "heebon.park@kt.com",
        "케이티텔보안네트워크": "bough38@naver.com",
        "커넥션네트워크": "bough38@naver.com",
        "애드라인": "bough38@naver.com",
        "제제컴퍼니": "bough38@naver.com",
        "아이티인프라": "bough38@naver.com"
    }

    smtp_server = "smtp.naver.com"
    smtp_port = 587

    header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    blue_font = Font(color="0000FF", bold=True)
    red_font = Font(color="FF0000", bold=True)
    thin_border = Border(left=Side(style='thin', color='000000'), 
                         right=Side(style='thin', color='000000'), 
                         top=Side(style='thin', color='000000'), 
                         bottom=Side(style='thin', color='000000'))
    thick_border = Border(left=Side(style='thick', color='000000'), 
                          right=Side(style='thick', color='000000'), 
                          top=Side(style='thick', color='000000'), 
                          bottom=Side(style='thick', color='000000'))

    for agency in agencies:
        summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
        
        if summary_filtered.empty:
            print(f"{agency}에 대한 데이터가 없습니다.")
            continue

        summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
        
        new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

        cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

        if summary_filtered['승인번호'].isnull().all():
            cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

        summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
        
        if agency == "(주)미래넷":
            excel_output_path = os.path.join(output_base_path, "(주)미래넷_4월정산.xlsx")
        else:
            excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

        with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
            summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
            new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
            if not cancel_detail_filtered.empty:
                cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
        
        wb = openpyxl.load_workbook(excel_output_path)
        
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thick_border
            
            if sheet == "표(요약)":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['U1'].font = Font(bold=True, color='0000FF')
                ws['U1'].number_format = '#,##0'
                ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='FF0000')
                ws['V1'].number_format = '#,##0'

            if sheet == "신규상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['C1'] = new_detail_filtered['계약번호'].nunique()
                ws['C1'].font = Font(bold=True)
                ws['C1'].number_format = '#,##0'
                ws['AB1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AB1'].font = Font(bold=True, color='0000FF')
                ws['AB1'].number_format = '#,##0'

            if sheet == "해지상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['E1'] = cancel_detail_filtered['계약번호'].nunique()
                ws['E1'].font = Font(bold=True)
                ws['E1'].number_format = '#,##0'
                ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AF1'].font = Font(bold=True, color='FF0000')
                ws['AF1'].number_format = '#,##0'
        
        wb.save(excel_output_path)

        receiver_email = dealer_info[agency]
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

        summary_html = f"""
            <p><b>표(요약):</b></p>
            <p>ㅇ 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</p>
            <p>ㅇ 최종지급액: <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            {f"<p>ㅇ 해지환수금액: <span style='color: red; font-weight: bold;'>{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>" if summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum() > 0 else ''}
            {f"<p>ㅇ 승인번호: <span style='color: green; font-weight: bold;'>{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>" if not summary_filtered['승인번호'].dropna().empty else ''}
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
            </table>
        """

        new_detail_html = f"""
            <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
                </tr>
                  {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
            </table>
        """

        if not cancel_detail_filtered.empty:
            cancel_detail_html = f"""
                <p><b>해지상세:</b></p>
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
                </table>
                <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            """
        else:
            cancel_detail_html = ""

        body = f"""
        <!DOCTYPE html>
        <html>
        <body>
            <p>{agency}님, 안녕하세요.</p>
            <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
            <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
            {summary_html}
            {new_detail_html}
            {cancel_detail_html}
        </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        with open(excel_output_path, "rb") as attachment:
            part = MIMEApplication(attachment.read(), _subtype="octet-stream")
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
            msg.attach(part)

        # 이메일 전송 함수
        def send_email():
            try:
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()
                    server.login(sender_email, sender_password)
                    server.sendmail(sender_email, receiver_email, msg.as_string())
                print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")
            except smtplib.SMTPAuthenticationError:
                messagebox.showerror("오류", "이메일 또는 비밀번호가 잘못되었습니다. 다시 시도해주세요.")
            except Exception as e:
                messagebox.showerror("오류", f"이메일 전송 중 오류가 발생했습니다: {str(e)}")

        send_email()

    print("모든 파일이 처리되고 이메일이 전송되었습니다.")

if __name__ == "__main__":
    main()



2024-05-23 13:58:26.260 python[5124:82872] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


(주)미래넷에 대한 이메일을 heebon.park@kt.com로 전송했습니다.
구정례에 대한 이메일을 heebon.park@kt.com로 전송했습니다.
케이티텔보안네트워크에 대한 이메일을 bough38@naver.com로 전송했습니다.
커넥션네트워크에 대한 이메일을 bough38@naver.com로 전송했습니다.
애드라인에 대한 이메일을 bough38@naver.com로 전송했습니다.
제제컴퍼니에 대한 이메일을 bough38@naver.com로 전송했습니다.
아이티인프라에 대한 이메일을 bough38@naver.com로 전송했습니다.
모든 파일이 처리되고 이메일이 전송되었습니다.


In [1]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("대리점 및 이메일 설정")
        self.dealer_info = {
            "(주)미래넷": "heebon.park@kt.com",
            "구정례": "heebon.park@kt.com",
            "케이티텔보안네트워크": "bough38@naver.com",
            "커넥션네트워크": "bough38@naver.com",
            "애드라인": "bough38@naver.com",
            "제제컴퍼니": "bough38@naver.com",
            "아이티인프라": "bough38@naver.com"
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings')
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def main():
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()

    sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
    sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')

    df_summary = read_file_with_encoding(summary_path, header=6)
    df_new_detail = read_file_with_encoding(new_detail_path)
    df_cancel_detail = read_file_with_encoding(cancel_detail_path)

    df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
    df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

    required_columns = [
        '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
        '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
        '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
    ]

    if len(df_summary.columns) < len(required_columns):
        for col in required_columns:
            if col not in df_summary.columns:
                df_summary[col] = None

    df_summary = df_summary.fillna('')
    df_new_detail = df_new_detail.fillna('')
    df_cancel_detail = df_cancel_detail.fillna('')

    def fill_missing_values(df, target_row, rows_to_check, cols):
        for col in cols:
            for row in rows_to_check:
                if pd.isna(df.at[target_row, col]):
                    if not pd.isna(df.at[row, col]):
                        df.at[target_row, col] = df.at[row, col]
                        break

    columns_to_copy = df_summary.columns
    fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

    df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
    df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
    df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

    total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
    if not total_row_index.empty:
        df_summary = df_summary.drop(total_row_index)
        df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

    df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
    df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

    df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['환수금액'] = df_cancel_detail['환수금액'].replace(' - ', '0')
agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

    dealer_info = {
        "(주)미래넷": "heebon.park@kt.com",
        "구정례": "heebon.park@kt.com",
        "케이티텔보안네트워크": "bough38@naver.com",
        "커넥션네트워크": "bough38@naver.com",
        "애드라인": "bough38@naver.com",
        "제제컴퍼니": "bough38@naver.com",
        "아이티인프라": "bough38@naver.com"
    }

    smtp_server = "smtp.naver.com"
    smtp_port = 587

    header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    blue_font = Font(color="0000FF", bold=True)
    red_font = Font(color="FF0000", bold=True)
    thin_border = Border(left=Side(style='thin', color='000000'), 
                         right=Side(style='thin', color='000000'), 
                         top=Side(style='thin', color='000000'), 
                         bottom=Side(style='thin', color='000000'))
    thick_border = Border(left=Side(style='thick', color='000000'), 
                          right=Side(style='thick', color='000000'), 
                          top=Side(style='thick', color='000000'), 
                          bottom=Side(style='thick', color='000000'))

    for agency in agencies:
        summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
        
        if summary_filtered.empty:
            print(f"{agency}에 대한 데이터가 없습니다.")
            continue

        summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
        
        new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

        cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

        if summary_filtered['승인번호'].isnull().all():
            cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

        summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
        
        if agency == "(주)미래넷":
            excel_output_path = os.path.join(output_base_path, "(주)미래넷_4월정산.xlsx")
        else:
            excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

        with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
            summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
            new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
            if not cancel_detail_filtered.empty:
                cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
        
        wb = openpyxl.load_workbook(excel_output_path)
        
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thin_border
            
            if sheet == "표(요약)":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['U1'].font = Font(bold=True, color='0000FF')
                ws['U1'].number_format = '#,##0'
                ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='FF0000')
                ws['V1'].number_format = '#,##0'

            if sheet == "신규상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['C1'] = new_detail_filtered['계약번호'].nunique()
                ws['C1'].font = Font(bold=True)
                ws['C1'].number_format = '#,##0'
                ws['AB1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AB1'].font = Font(bold=True, color='0000FF')
                ws['AB1'].number_format = '#,##0'

            if sheet == "해지상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['E1'] = cancel_detail_filtered['계약번호'].nunique()
                ws['E1'].font = Font(bold=True)
                ws['E1'].number_format = '#,##0'
                ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AF1'].font = Font(bold=True, color='FF0000')
                ws['AF1'].number_format = '#,##0'
        
        wb.save(excel_output_path)

        receiver_email = dealer_info[agency]
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

        summary_html = f"""
            <p><b>표(요약):</b></p>
            <p>ㅇ 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</p>
            <p>ㅇ 최종지급액: <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            {f"<p>ㅇ 해지환수금액: <span style='color: red; font-weight: bold;'>{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>" if summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum() > 0 else ''}
            {f"<p>ㅇ 승인번호: <span style='color: green; font-weight: bold;'>{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>" if not summary_filtered['승인번호'].dropna().empty else ''}
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
            </table>
        """

        new_detail_html = f"""
            <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
            </table>
        """

        if not cancel_detail_filtered.empty:
            cancel_detail_html = f"""
                <p><b>해지상세:</b> <span style="color: red; font-weight: bold;">환수금액 합계: {cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
                </table>
            """
        else:
            cancel_detail_html = ""

        body = f"""
        <!DOCTYPE html>
        <html>
        <body style="font-size: 12px;">
            <p>{agency}님, 안녕하세요.</p>
            <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
            <p>대리점 4월 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
            {summary_html}
            {new_detail_html}
            {cancel_detail_html}
        </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        with open(excel_output_path, "rb") as attachment:
            part = MIMEApplication(attachment.read(), _subtype="octet-stream")
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
            msg.attach(part)

        # 이메일 전송 함수
        def send_email():
            try:
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()
                    server.login(sender_email, sender_password)
                    server.sendmail(sender_email, receiver_email, msg.as_string())
                messagebox.showinfo("성공", f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")
                root.quit()
            except smtplib.SMTPAuthenticationError:
                messagebox.showerror("오류", "이메일 또는 비밀번호가 잘못되었습니다. 다시 시도해주세요.")
            except Exception as e:
                messagebox.showerror("오류", f"이메일 전송 중 오류가 발생했습니다: {str(e)}")

        send_email()

    print("모든 파일이 처리되고 이메일이 전송되었습니다.")

if __name__ == "__main__":
    main()


IndentationError: unexpected indent (2877886691.py, line 185)

In [ ]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("대리점 및 이메일 설정")
        self.dealer_info = {
            "(주)미래넷": "heebon.park@kt.com",
            "구정례": "heebon.park@kt.com",
            "케이티텔보안네트워크": "bough38@naver.com",
            "커넥션네트워크": "bough38@naver.com",
            "애드라인": "bough38@naver.com",
            "제제컴퍼니": "bough38@naver.com",
            "아이티인프라": "bough38@naver.com"
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings', selectmode='extended')
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

        self.month_label = tk.Label(self.root, text="정산월:")
        self.month_label.pack()
        self.month_entry = tk.Entry(self.root)
        self.month_entry.pack()

        send_button_frame = tk.Frame(self.root)
        send_button_frame.pack(pady=10)
        tk.Button(send_button_frame, text="전체 발송", command=self.send_all).pack(side=tk.LEFT, padx=5)
        tk.Button(send_button_frame, text="선택 발송", command=self.send_selected).pack(side=tk.LEFT, padx=5)

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

    def get_selected_agencies(self):
        selected_items = self.tree.selection()
        return [self.tree.item(item)['values'][0] for item in selected_items]

    def send_all(self):
        self.send_emails(self.dealer_info.keys())

    def send_selected(self):
        selected_agencies = self.get_selected_agencies()
        if selected_agencies:
            self.send_emails(selected_agencies)
        else:
            messagebox.showerror("오류", "선택된 대리점이 없습니다.")

    def send_emails(self, agencies):
        sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
        sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')
        month = self.month_entry.get().strip()

        if not sender_email or not sender_password or not month:
            messagebox.showerror("오류", "이메일, 비밀번호, 정산월을 모두 입력해주세요.")
            return

        # 파일 경로 설정
        base_path = "/Users/heebonpark/Downloads"
        output_base_path = os.path.join(base_path, f"{month} 정산 전송")
        os.makedirs(output_base_path, exist_ok=True)

        summary_path = os.path.join(base_path, f"2024.{month}_표(요약).cs")
        new_detail_path = os.path.join(base_path, f"2024.{month}_신규상세.cs")
        cancel_detail_path = os.path.join(base_path, f"2024.{month}_해지상세.cs")

        df_summary = read_file_with_encoding(summary_path, header=6)
        df_new_detail = read_file_with_encoding(new_detail_path)
        df_cancel_detail = read_file_with_encoding(cancel_detail_path)

        df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
        df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

        required_columns = [
            '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
            '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
            '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
        ]

        if len(df_summary.columns) < len(required_columns):
            for col in required_columns:
                if col not in df_summary.columns:
                    df_summary[col] = None

        df_summary = df_summary.fillna('')
        df_new_detail = df_new_detail.fillna('')
        df_cancel_detail = df_cancel_detail.fillna('')

        def fill_missing_values(df, target_row, rows_to_check, cols):
            for col in cols:
                for row in rows_to_check:
                    if pd.isna(df.at[target_row, col]):
                        if not pd.isna(df.at[row, col]):
                            df.at[target_row, col] = df.at[row, col]
                            break

        columns_to_copy = df_summary.columns
        fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

        df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
        df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
        df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

        total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
        if not total_row_index.empty:
                       df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

        df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
        df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
        df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

        df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
        df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
        df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

        smtp_server = "smtp.naver.com"
        smtp_port = 587

        header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
        header_font = Font(color="FFFFFF", bold=True)
        blue_font = Font(color="0000FF", bold=True)
        red_font = Font(color="FF0000", bold=True)
        thin_border = Border(left=Side(style='thin', color='000000'), 
                            right=Side(style='thin', color='000000'), 
                            top=Side(style='thin', color='000000'), 
                            bottom=Side(style='thin', color='000000'))

        for agency in agencies:
            summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
            
            if summary_filtered.empty:
                print(f"{agency}에 대한 데이터가 없습니다.")
                continue

            summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
            
            new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
            new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

            cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
            cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

            if summary_filtered['승인번호'].isnull().all():
                cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

            summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
            
            if agency == "(주)미래넷":
                excel_output_path = os.path.join(output_base_path, f"(주)미래넷_{month}정산.xlsx")
            else:
                excel_output_path = os.path.join(output_base_path, f"{agency}_{month}정산.xlsx")

            with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
                summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
                new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
                if not cancel_detail_filtered.empty:
                    cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
            
            wb = openpyxl.load_workbook(excel_output_path)
            
            for sheet in wb.sheetnames:
                ws = wb[sheet]
                
                for cell in ws[1]:
                    cell.fill = header_fill
                    cell.font = header_font
                    cell.border = thin_border
                
                if sheet == "표(요약)":
                    ws.insert_rows(1)
                    ws['A1'] = '합계'
                    ws['A1'].font = Font(bold=True)
                    ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()
                    ws['U1'].font = Font(bold=True, color='0000FF')
                    ws['U1'].number_format = '#,##0'
                    ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()
                    ws['V1'].font = Font(bold=True, color='FF0000')
                    ws['V1'].number_format = '#,##0'

                if sheet == "신규상세":
                    ws.insert_rows(1)
                    ws['A1'] = '합계'
                    ws['A1'].font = Font(bold=True)
                    ws['C1'] = new_detail_filtered['계약번호'].nunique()
                    ws['C1'].font = Font(bold=True)
                    ws['C1'].number_format = '#,##0'
                    ws['AB1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()
                    ws['AB1'].font = Font(bold=True, color='0000FF')
                    ws['AB1'].number_format = '#,##0'

                if sheet == "해지상세":
                    ws.insert_rows(1)
                    ws['A1'] = '합계'
                    ws['A1'].font = Font(bold=True)
                    ws['E1'] = cancel_detail_filtered['계약번호'].nunique()
                    ws['E1'].font = Font(bold=True)
                    ws['E1'].number_format = '#,##0'
                    ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()
                    ws['AF1'].font = Font(bold=True, color='FF0000')
                    ws['AF1'].number_format = '#,##0'
            
            wb.save(excel_output_path)

            receiver_email = dealer_info[agency]
            msg = MIMEMultipart()
            msg['From'] = sender_email
            msg['To'] = receiver_email
            msg['Subject'] = f"{month} 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

            summary_html = f"""
                <p><b>표(요약):</b></p>
                <p>ㅇ 합계액(수수료): {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</p>
                <p>ㅇ 최종지급액(유지수수료포함): <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
                {f"<p>ㅇ 해지환수금액: <span style='color: red; font-weight: bold;'>{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>" if summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum() > 0 else ''}
                {f"<p>ㅇ 승인번호: <span style='color: green; font-weight: bold;'>{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>" if not summary_filtered['승인번호'].dropna().empty else ''}
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
                </table>
            """

            new_detail_html = f"""
                <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
                </table>
            """

            if not cancel_detail_filtered.empty:
                cancel_detail_html = f"""
                    <p><b>해지상세:</b></p>
                                      <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                        <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                            {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
                        </tr>
                        {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
                    </table>
                    <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
                """
            else:
                cancel_detail_html = ""

            body = f"""
            <!DOCTYPE html>
            <html>
            <body style="font-size: 12px;">
                <p>{agency}님, 안녕하세요.</p>
                <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
                <p>대리점 {month} 정산실적을 아래와 같이 송부드리니 {month} 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
                {summary_html}
                {new_detail_html}
                {cancel_detail_html}
            </body>
            </html>
            """

            msg.attach(MIMEText(body, 'html'))

            with open(excel_output_path, "rb") as attachment:
                part = MIMEApplication(attachment.read(), _subtype="octet-stream")
                part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
                msg.attach(part)

            def send_email():
                try:
                    with smtplib.SMTP(smtp_server, smtp_port) as server:
                        server.starttls()
                        server.login(sender_email, sender_password)
                        server.sendmail(sender_email, receiver_email, msg.as_string())
                    messagebox.showinfo("성공", f"{agency}에 대한 이메일이 {receiver_email}로 전송되었습니다.")
                except smtplib.SMTPAuthenticationError:
                    messagebox.showerror("오류", "이메일 또는 비밀번호가 잘못되었습니다. 다시 시도해주세요.")
                except Exception as e:
                    messagebox.showerror("오류", f"이메일 전송 중 오류가 발생했습니다: {str(e)}")

            send_email()

        messagebox.showinfo("완료", "모든 파일이 처리되고 이메일이 전송되었습니다.")
        self.root.quit()

def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

if __name__ == "__main__":
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()


2024-05-23 14:14:31.983 python[5492:92000] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/heebonpark/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_5492/3530436188.py", line 118, in send_all
    self.send_emails(self.dealer_info.keys())
  File "/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_5492/3530436188.py", line 145, in send_emails
    df_summary = read_file_with_encoding(summary_path, header=6)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/_f/8db5fbdx6_j62bmzy_26gf180000gn/T/ipykernel_5492/3530436188.py", line 366, in read_file_with_encoding
    raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
FileNotFoundError: /Users/heebonpark/Downloads/2024.5월_표(요약).cs 파일이

In [2]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, filedialog, ttk

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("대리점 및 이메일 설정")
        self.dealer_info = {
            "(주)미래넷": "heebon.park@kt.com",
            "구정례": "heebon.park@kt.com",
            "케이티텔보안네트워크": "bough38@naver.com",
            "커넥션네트워크": "bough38@naver.com",
            "애드라인": "bough38@naver.com",
            "제제컴퍼니": "bough38@naver.com",
            "아이티인프라": "bough38@naver.com"
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings')
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

        file_button_frame = tk.Frame(self.root)
        file_button_frame.pack(pady=10)
        tk.Button(file_button_frame, text="표(요약) 파일 선택", command=lambda: self.select_file("summary")).pack(side=tk.LEFT, padx=5)
        tk.Button(file_button_frame, text="신규상세 파일 선택", command=lambda: self.select_file("new_detail")).pack(side=tk.LEFT, padx=5)
        tk.Button(file_button_frame, text="해지상세 파일 선택", command=lambda: self.select_file("cancel_detail")).pack(side=tk.LEFT, padx=5)

        self.file_paths = {"summary": "", "new_detail": "", "cancel_detail": ""}

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

    def select_file(self, file_type):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.file_paths[file_type] = file_path
            messagebox.showinfo("파일 선택", f"{file_type} 파일이 선택되었습니다.")

def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

def main():
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()

    sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
    sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')

    summary_path = app.file_paths["summary"]
    new_detail_path = app.file_paths["new_detail"]
    cancel_detail_path = app.file_paths["cancel_detail"]

    df_summary = read_file_with_encoding(summary_path, header=6)
    df_new_detail = read_file_with_encoding(new_detail_path)
    df_cancel_detail = read_file_with_encoding(cancel_detail_path)

    df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
    df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

    required_columns = [
        '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
        '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
        '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
    ]

    if len(df_summary.columns) < len(required_columns):
        for col in required_columns:
            if col not in df_summary.columns:
                df_summary[col] = None

    df_summary = df_summary.fillna('')
    df_new_detail = df_new_detail.fillna('')
    df_cancel_detail = df_cancel_detail.fillna('')

    def fill_missing_values(df, target_row, rows_to_check, cols):
        for col in cols:
            for row in rows_to_check:
                if pd.isna(df.at[target_row, col]):
                    if not pd.isna(df.at[row, col]):
                        df.at[target_row, col] = df.at[row, col]
                        break

    columns_to_copy = df_summary.columns
    fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

    df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
    df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
    df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

    total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
    if not total_row_index.empty:
        df_summary = df_summary.drop(total_row_index)
        df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

    df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
    df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

    df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

    # 파일 경로 설정
    base_path = filedialog.askdirectory(title="파일 저장 경로 선택")
    if not base_path:
        messagebox.showerror("오류", "파일 저장 경로가 선택되지 않았습니다.")
        return
    output_base_path = os.path.join(base_path, "정산 전송")
    os.makedirs(output_base_path, exist_ok=True)

    agencies = list(app.dealer_info.keys())
    dealer_info = app.dealer_info

    smtp_server = "smtp.naver.com"
    smtp_port = 587

    header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    blue_font = Font(color="0000FF", bold=True)
    red_font = Font(color="FF0000", bold=True)
    thin_border = Border(left=Side(style='thin', color='000000'), 
                         right=Side(style='thin', color='000000'), 
                         top=Side(style='thin', color='000000'), 
                         bottom=Side(style='thin', color='000000'))
    thick_border = Border(left=Side(style='thick', color='000000'), 
                          right=Side(style='thick', color='000000'), 
                          top=Side(style='thick', color='000000'), 
                          bottom=Side(style='thick', color='000000'))

    for agency in agencies:
        summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
        
        if summary_filtered.empty:
            print(f"{agency}에 대한 데이터가 없습니다.")
            continue

        summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
        
        new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

        cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

        if summary_filtered['승인번호'].isnull().all():
            cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

        summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')

        # 정산년월 가져오기
        settlement_year_month = summary_filtered['정산년월(YYMM)'].iloc[0]
        year = settlement_year_month[:2]
        month = settlement_year_month[2:]
        year_month_text = f"20{year}년 {month}월"

        if agency == "(주)미래넷":
            excel_output_path = os.path.join(output_base_path, f"{year_month_text}_{agency}_정산.xlsx")
        else:
            excel_output_path = os.path.join(output_base_path, f"{year_month_text}_{agency}_정산.xlsx")

        with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
            summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
            new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
            if not cancel_detail_filtered.empty:
                cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
        
        wb = openpyxl.load_workbook(excel_output_path)
        
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thin_border
            
            if sheet == "표(요약)":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['U1'].font = Font(bold=True, color='0000FF')
                ws['U1'].number_format = '#,##0'
                ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='FF0000')
                ws['V1'].number_format = '#,##0'

            if sheet == "신규상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['C1'] = new_detail_filtered['계약번호'].nunique()
                ws['C1'].font = Font(bold=True)
                ws['C1'].number_format = '#,##0'
                ws['AB1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AB1'].font = Font(bold=True, color='0000FF')
                ws['AB1'].number_format = '#,##0'

            if sheet == "해지상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['E1'] = cancel_detail_filtered['계약번호'].nunique()
                ws['E1'].font = Font(bold=True)
                ws['E1'].number_format = '#,##0'
                ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AF1'].font = Font(bold=True, color='FF0000')
                ws['AF1'].number_format = '#,##0'
        
        wb.save(excel_output_path)

        receiver_email = dealer_info[agency]
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = f"{year_month_text} 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

        summary_html = f"""
            <p><b>표(요약):</b></p>
            <p>ㅇ 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</p>
            <p>ㅇ 최종지급액: <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            {f"<p>ㅇ 해지환수금액: <span style='color: red; font-weight: bold;'>{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>" if summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum() > 0 else ''}
            {f"<p>ㅇ 승인번호: <span style='color: green; font-weight: bold;'>{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>" if not summary_filtered['승인번호'].dropna().empty else ''}
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px; border-collapse: collapse;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th style="border: 1px solid #000;">{col}</th>' for col in summary_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td style=\'border: 1px solid #000;\'>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
            </table>
        """

        new_detail_html = f"""
            <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float
            .sum():,.0f}</span></p>
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px; border-collapse: collapse;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th style="border: 1px solid #000;">{col}</th>' for col in new_detail_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td style=\'border: 1px solid #000;\'>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
            </table>
        """

        if not cancel_detail_filtered.empty:
            cancel_detail_html = f"""
                <p><b>해지상세: 환수금액 합계 <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></b></p>
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px; border-collapse: collapse;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th style="border: 1px solid #000;">{col}</th>' for col in cancel_detail_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td style=\'border: 1px solid #000;\'>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
                </table>
            """
        else:
            cancel_detail_html = ""

        body = f"""
        <!DOCTYPE html>
        <html>
        <body style="font-size: 12px;">
            <p>{agency}님, 안녕하세요.</p>
            <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
            <p>대리점 {year_month_text} 정산실적을 아래와 같이 송부드리니 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
            {summary_html}
            {new_detail_html}
            {cancel_detail_html}
        </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        with open(excel_output_path, "rb") as attachment:
            part = MIMEApplication(attachment.read(), _subtype="octet-stream")
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
            msg.attach(part)

        # 이메일 전송 함수
        def send_email():
            try:
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()
                    server.login(sender_email, sender_password)
                    server.sendmail(sender_email, receiver_email, msg.as_string())
                messagebox.showinfo("성공", f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")
                root.quit()
            except smtplib.SMTPAuthenticationError:
                messagebox.showerror("오류", "이메일 또는 비밀번호가 잘못되었습니다. 다시 시도해주세요.")
            except Exception as e:
                messagebox.showerror("오류", f"이메일 전송 중 오류가 발생했습니다: {str(e)}")

        send_email()

if __name__ == "__main__":
    main()



SyntaxError: f-string expression part cannot include a backslash (894807615.py, line 317)

In [4]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk, filedialog

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("대리점 및 이메일 설정")
        self.dealer_info = {
            "(주)미래넷": "heebon.park@kt.com",
            "구정례": "heebon.park@kt.com",
            "케이티텔보안네트워크": "bough38@naver.com",
            "커넥션네트워크": "bough38@naver.com",
            "애드라인": "bough38@naver.com",
            "제제컴퍼니": "bough38@naver.com",
            "아이티인프라": "bough38@naver.com"
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings', selectmode="extended")
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def main():
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()

    # 파일 경로 설정
    base_path = filedialog.askdirectory(title="폴더 선택")
    output_base_path = os.path.join(base_path, "정산 전송")
    os.makedirs(output_base_path, exist_ok=True)

    summary_path = filedialog.askopenfilename(title="표(요약) 파일 선택", filetypes=[("CSV Files", "*.cs")])
    new_detail_path = filedialog.askopenfilename(title="신규상세 파일 선택", filetypes=[("CSV Files", "*.cs")])
    cancel_detail_path = filedialog.askopenfilename(title="해지상세 파일 선택", filetypes=[("CSV Files", "*.cs")])

    sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
    sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')
    year_month = simpledialog.askstring("Input", "정산 연월을 입력하세요 (예: 2404):")
    year_month_text = f"20{year_month[:2]}년 {year_month[2:]}월"

    df_summary = read_file_with_encoding(summary_path, header=6)
    df_new_detail = read_file_with_encoding(new_detail_path)
    df_cancel_detail = read_file_with_encoding(cancel_detail_path)

    df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
    df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

    required_columns = [
        '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
        '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
        '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
    ]

    if len(df_summary.columns) < len(required_columns):
        for col in required_columns:
            if col not in df_summary.columns:
                df_summary[col] = None

    df_summary = df_summary.fillna('')
    df_new_detail = df_new_detail.fillna('')
    df_cancel_detail = df_cancel_detail.fillna('')

    def fill_missing_values(df, target_row, rows_to_check, cols):
        for col in cols:
            for row in rows_to_check:
                if pd.isna(df.at[target_row, col]):
                    if not pd.isna(df.at[row, col]):
                        df.at[target_row, col] = df.at[row, col]
                        break

    columns_to_copy = df_summary.columns
    fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

    df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
    df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
    df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

    total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
    if not total_row_index.empty:
        df_summary = df_summary.drop(total_row_index)
        df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

    df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
    df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

    df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
        df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]

    agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

    smtp_server = "smtp.naver.com"
    smtp_port = 587

    header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    blue_font = Font(color="0000FF", bold=True)
    red_font = Font(color="FF0000", bold=True)
    thin_border = Border(left=Side(style='thin', color='000000'), 
                         right=Side(style='thin', color='000000'), 
                         top=Side(style='thin', color='000000'), 
                         bottom=Side(style='thin', color='000000'))
    thick_border = Border(left=Side(style='thick', color='000000'), 
                          right=Side(style='thick', color='000000'), 
                          top=Side(style='thick', color='000000'), 
                          bottom=Side(style='thick', color='000000'))

    for agency in agencies:
        summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
        
        if summary_filtered.empty:
            print(f"{agency}에 대한 데이터가 없습니다.")
            continue

        summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
        
        new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

        cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

        if summary_filtered['승인번호'].isnull().all():
            cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

        summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
        
        excel_output_path = os.path.join(output_base_path, f"{year_month_text}_{agency}_대리점_정산.xlsx")

        with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
            summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
            new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
            if not cancel_detail_filtered.empty:
                cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
        
        wb = openpyxl.load_workbook(excel_output_path)
        
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thick_border
            
            if sheet == "표(요약)":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['U1'].font = Font(bold=True, color='0000FF')
                ws['U1'].number_format = '#,##0'
                ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='FF0000')
                ws['V1'].number_format = '#,##0'

            if sheet == "신규상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['C1'] = new_detail_filtered['계약번호'].nunique()
                ws['C1'].font = Font(bold=True)
                ws['C1'].number_format = '#,##0'
                ws['AB1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AB1'].font = Font(bold=True, color='0000FF')
                ws['AB1'].number_format = '#,##0'

            if sheet == "해지상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['E1'] = cancel_detail_filtered['계약번호'].nunique()
                ws['E1'].font = Font(bold=True)
                ws['E1'].number_format = '#,##0'
                ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AF1'].font = Font(bold=True, color='FF0000')
                ws['AF1'].number_format = '#,##0'
        
        wb.save(excel_output_path)

        receiver_email = app.dealer_info[agency]
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = f"{year_month_text} 정산실적_{agency}_대리점"

        summary_html = f"""
            <p><b>표(요약):</b></p>
            <p>ㅇ 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</p>
            <p>ㅇ 최종지급액: <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            {f"<p>ㅇ 해지환수금액: <span style='color: red; font-weight: bold;'>{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>" if summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum() > 0 else ''}
            {f"<p>ㅇ 승인번호: <span style='color: green; font-weight: bold;'>{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>" if not summary_filtered['승인번호'].dropna().empty else ''}
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
            </table>
        """

        new_detail_html = f"""
            <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
            </table>
        """

               if not cancel_detail_filtered.empty:
            cancel_detail_html = f"""
                <p><b>해지상세:</b></p>
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
                </table>
                <p><b>환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            """
        else:
            cancel_detail_html = ""

        body = f"""
        <!DOCTYPE html>
        <html>
        <body style="font-size: 12px;">
            <p>{agency}님, 안녕하세요.</p>
            <p>kt텔레캅 사외채널 담당자 장재혁입니다.</p>
            <p>대리점 {year_month_text} 정산실적을 아래와 같이 송부드리니 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
            {summary_html}
            {new_detail_html}
            {cancel_detail_html}
        </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        with open(excel_output_path, "rb") as attachment:
            part = MIMEApplication(attachment.read(), _subtype="octet-stream")
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
            msg.attach(part)

        # 이메일 전송 함수
        def send_email():
            try:
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()
                    server.login(sender_email, sender_password)
                    server.sendmail(sender_email, receiver_email, msg.as_string())
                print(f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")
                messagebox.showinfo("성공", f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")
            except smtplib.SMTPAuthenticationError:
                messagebox.showerror("오류", "이메일 또는 비밀번호가 잘못되었습니다. 다시 시도해주세요.")
            except Exception as e:
                messagebox.showerror("오류", f"이메일 전송 중 오류가 발생했습니다: {str(e)}")

        send_email()

    print("모든 파일이 처리되고 이메일이 전송되었습니다.")

if __name__ == "__main__":
    main()

    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 307)

In [1]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("대리점 및 이메일 설정")
        self.dealer_info = {
            "(주)미래넷": "heebon.park@kt.com",
            "구정례": "heebon.park@kt.com",
            "케이티텔보안네트워크": "bough38@naver.com",
            "커넥션네트워크": "bough38@naver.com",
            "애드라인": "bough38@naver.com",
            "제제컴퍼니": "bough38@naver.com",
            "아이티인프라": "bough38@naver.com"
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings')
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

# 파일 경로 설정
base_path = "/Users/heebonpark/Downloads"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_표(요약).cs")
new_detail_path = os.path.join(base_path, "2024.04월_신규상세.cs")
cancel_detail_path = os.path.join(base_path, "2024.04월_해지상세.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def main():
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()

    sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
    sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')

    df_summary = read_file_with_encoding(summary_path, header=6)
    df_new_detail = read_file_with_encoding(new_detail_path)
    df_cancel_detail = read_file_with_encoding(cancel_detail_path)

    df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
    df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

    required_columns = [
        '정산년월(YYMM)', '지사', '추천자사번', '대표자', '상호명', '사업자등록번호',
        '신규월정료', '신규건수', '모집수수료', '성사율수수료', '완결비중수수료', '완결영업정책수수료', '합계',
        '유지수수료', '판매형', '소급액', '전월미지급액', '전월미환수', '공급가액', '부가세(공급가액10%)', '최종지급액', '해지환수금액(별도세금계산서발행)', '승인번호'
    ]

    if len(df_summary.columns) < len(required_columns):
        for col in required_columns:
            if col not in df_summary.columns:
                df_summary[col] = None

    df_summary = df_summary.fillna('')
    df_new_detail = df_new_detail.fillna('')
    df_cancel_detail = df_cancel_detail.fillna('')

    def fill_missing_values(df, target_row, rows_to_check, cols):
        for col in cols:
            for row in rows_to_check:
                if pd.isna(df.at[target_row, col]):
                    if not pd.isna(df.at[row, col]):
                        df.at[target_row, col] = df.at[row, col]
                        break

    columns_to_copy = df_summary.columns
    fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

    df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
    df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
    df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

    total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
    if not total_row_index.empty:
        df_summary = df_summary.drop(total_row_index)
        df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

    df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
    df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

    df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]
    agencies = ["(주)미래넷", "구정례", "케이티텔보안네트워크", "커넥션네트워크", "애드라인", "제제컴퍼니", "아이티인프라"]

    dealer_info = {
        "(주)미래넷": "heebon.park@kt.com",
        "구정례": "heebon.park@kt.com",
        "케이티텔보안네트워크": "bough38@naver.com",
        "커넥션네트워크": "bough38@naver.com",
        "애드라인": "bough38@naver.com",
        "제제컴퍼니": "bough38@naver.com",
        "아이티인프라": "bough38@naver.com"
    }

    smtp_server = "smtp.naver.com"
    smtp_port = 587

    header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    blue_font = Font(color="0000FF", bold=True)
    red_font = Font(color="FF0000", bold=True)
    thin_border = Border(left=Side(style='thin', color='000000'), 
                         right=Side(style='thin', color='000000'), 
                         top=Side(style='thin', color='000000'), 
                         bottom=Side(style='thin', color='000000'))
    thick_border = Border(left=Side(style='thick', color='000000'), 
                          right=Side(style='thick', color='000000'), 
                          top=Side(style='thick', color='000000'), 
                          bottom=Side(style='thick', color='000000'))

    for agency in agencies:
        summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
        
        if summary_filtered.empty:
            print(f"{agency}에 대한 데이터가 없습니다.")
            continue

        summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
        
        new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

        cancel_detail_filtered = df_cancel_detail[df_cancel_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        cancel_detail_filtered = cancel_detail_filtered.drop(columns=["대리점 해지 여부"], errors='ignore')

        if summary_filtered['승인번호'].isnull().all():
            cancel_detail_filtered = pd.DataFrame(columns=cancel_detail_filtered.columns)

        summary_filtered['해지환수금액(별도세금계산서발행)'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace(' - ', '0')
        
        if agency == "(주)미래넷":
            excel_output_path = os.path.join(output_base_path, "(주)미래넷_4월정산.xlsx")
        else:
            excel_output_path = os.path.join(output_base_path, f"{agency}_4월정산.xlsx")

        with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
            summary_filtered.to_excel(writer, sheet_name="표(요약)", index=False)
            new_detail_filtered.to_excel(writer, sheet_name="신규상세", index=False)
            if not cancel_detail_filtered.empty:
                cancel_detail_filtered.to_excel(writer, sheet_name="해지상세", index=False)
        
        wb = openpyxl.load_workbook(excel_output_path)
        
        for sheet in wb.sheetnames:
            ws = wb[sheet]
            
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thin_border
            
            if sheet == "표(요약)":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['U1'] = summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['U1'].font = Font(bold=True, color='0000FF')
                ws['U1'].number_format = '#,##0'
                ws['V1'] = summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum()
                ws['V1'].font = Font(bold=True, color='FF0000')
                ws['V1'].number_format = '#,##0'

            if sheet == "신규상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['C1'] = new_detail_filtered['계약번호'].nunique()
                ws['C1'].font = Font(bold=True)
                ws['C1'].number_format = '#,##0'
                ws['AB1'] = new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AB1'].font = Font(bold=True, color='0000FF')
                ws['AB1'].number_format = '#,##0'

            if sheet == "해지상세":
                ws.insert_rows(1)
                ws['A1'] = '합계'
                ws['A1'].font = Font(bold=True)
                ws['E1'] = cancel_detail_filtered['계약번호'].nunique()
                ws['E1'].font = Font(bold=True)
                ws['E1'].number_format = '#,##0'
                ws['AF1'] = cancel_detail_filtered['환수금액'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AF1'].font = Font(bold=True, color='FF0000')
                ws['AF1'].number_format = '#,##0'
        
        wb.save(excel_output_path)

        receiver_email = dealer_info[agency]
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

        summary_html = f"""
            <p><b>표(요약):</b></p>
            <p>ㅇ 합계액: {summary_filtered['합계'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</p>
            <p>ㅇ 최종지급액: <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            {f"<p>ㅇ 해지환수금액: <span style='color: red; font-weight: bold;'>{summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>" if summary_filtered['해지환수금액(별도세금계산서발행)'].replace({',': ''}, regex=True).astype(float).sum() > 0 else ''}
            {f"<p>ㅇ 승인번호: <span style='color: green; font-weight: bold;'>{', '.join(summary_filtered['승인번호'].dropna().astype(str).unique())}</span></p>" if not summary_filtered['승인번호'].dropna().empty else ''}
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
            </table>
        """

        new_detail_html = f"""
            <p><b>신규상세:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계: {new_detail_filtered['지급수수료'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
                </tr>
                  {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
            </table>
        """

        if not cancel_detail_filtered.empty:
            cancel_detail_html = f"""
                <p><b>[해지상세]</b></p>
                <p><b>ㅇ 환수금액 합계:</b> <span style="color:red; font-weight:bold;">{cancel_detail_filtered["환수금액"].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
                <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                    <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                        {''.join([f'<th>{col}</th>' for col in cancel_detail_filtered.columns])}
                    </tr>
                    {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in cancel_detail_filtered.values])}
                </table>
                """
        else:
            cancel_detail_html = ""

        body = f"""
        <!DOCTYPE html>
        <html>
        <body>
            <p style="font-size:12px;">{agency}님, 안녕하세요.</p>
            <p style="font-size:12px;">kt텔레캅 사외채널 담당자 장재혁입니다.</p>
            <p style="font-size:12px;">대리점 정산실적을 아래와 같이 송부드리니 5월 6일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
            {summary_html}
            {new_detail_html}
            {cancel_detail_html}
        </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        with open(excel_output_path, "rb") as attachment:
            part = MIMEApplication(attachment.read(), _subtype="octet-stream")
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
            msg.attach(part)

        # 이메일 전송 함수
        def send_email():
            try:
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()
                    server.login(sender_email, sender_password)
                    server.sendmail(sender_email, receiver_email, msg.as_string())
                messagebox.showinfo("성공", f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")
            except smtplib.SMTPAuthenticationError:
                messagebox.showerror("오류", "이메일 또는 비밀번호가 잘못되었습니다. 다시 시도해주세요.")
            except Exception as e:
                messagebox.showerror("오류", f"이메일 전송 중 오류가 발생했습니다: {str(e)}")

        send_email()

    print("모든 파일이 처리되고 이메일이 전송되었습니다.")

if __name__ == "__main__":
    main()

   


2024-05-24 22:23:03.004 python[28678:627718] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


모든 파일이 처리되고 이메일이 전송되었습니다.


In [1]:
pyinstaller --onefile --windowed 대리점메일전송.py


SyntaxError: invalid syntax (2671233971.py, line 1)